# Geological Interpretor Development

This is a notebook for testing and developping some of the basic code in this package.

### Some initial imports

In [ ]:
import numpy as np
import owlready2 as owl
class MalissiaBaseError(Exception):
    """This is the base class for all exceptions of this package"""

In [ ]:
owl.onto_path.append("../ontologies/")
mogi = owl.get_ontology("mogi.owl").load()
mogi

## Geological Knowledge Manager

**GeologicalKnowledgeManager** may know different instances of **GeologicalKnowledgeFramework**,<br>
for example to allow differenciating scenarios or for allowing customisation of knowledge and its formalisation.

**GeologicalKnowledgeFramework** provides access to concept definitions for providing knowledge.

In [ ]:
import os

class GeologicalKnowledgeManager(object):
    """GeologicalKnowledgeManager is managing one or several GeologicalKnowledgeFramework.
    
    The GeologicalKnowledgeManager is typically a singleton, so there is always one and only one instance of it.
    
    The GeologicalKnowledgeManager may know different instances of GeologicalKnowledgeFramework,
    for example to allow different interpretation scenarios or for allowing user-specific customisation
    of knowledge and its formalisation.
    
    GeologicalKnowledgeFramework are typically ontologies and extensions defined in this package or elsewhere.
    """
    
    def __new__(cls):
        """Method to access (and create if needed) the only allowed instance of this class.
        
        Returns:
        - an instance of GeologicalKnowledgeManager"""
        if not hasattr(cls, 'instance'):
            cls.instance = super(GeologicalKnowledgeManager, cls).__new__(cls)
            cls.initialised= False
        return cls.instance
        
    def __init__(self, default= "mogi", default_source_directory= "../ontologies/", default_source_file= "mogi.owl", default_ontology_backend= "owlready2"):
        """Initializes the GeologicalKnowledgeManager with some default values from configuration.
        
        Parameters:
        - default: specifies the name of the default knowledge framework
        - default_source_directory: specifies the default folder containing of the knowledge framework definitions
        - default_source_file: file contained in the source_directory defining the knowledge framework (e.g., .owl file)
        - default_ontology_backend: specifies the default ontology backend to be used
        """
        if not self.initialised:
            self._initialise(default= default, default_source_directory= default_source_directory, default_source_file= default_source_file, default_ontology_backend= default_ontology_backend)
            
    def _initialise(self, default, default_source_directory, default_source_file, default_ontology_backend):
        """Initializes the GeologicalKnowledgeManager with some default values from configuration.
        
        Parameters:
        - default: specifies the name of the default knowledge framework
        - default_source_directory: specifies the default folder containing of the knowledge framework definitions
        - default_source_file: file contained in the source_directory defining the knowledge framework (e.g., .owl file)
        - default_ontology_backend: specifies the default ontology backend to be used
        """
        self.default= default
        self.default_source_directory= default_source_directory
        self.default_source_file= default_source_file
        self.default_ontology_backend= default_ontology_backend
        
        self.knowledge_framework_dict = {}
        
        self.initialised= True
        
    def reset(self, default= "mogi", default_source_directory= "../ontologies/", default_source_file= "mogi.owl", default_ontology_backend= "owlready2"):
        """Reinitializes the GeologicalKnowledgeManager with some default values from configuration.
        
        Parameters:
        - default: specifies the name of the default knowledge framework
        - default_source_directory: specifies the default folder containing of the knowledge framework definitions
        - default_source_file: file contained in the source_directory defining the knowledge framework (e.g., .owl file)
        - default_ontology_backend: specifies the default ontology backend to be used
        """
        self._initialise(default= default, default_source_directory= default_source_directory, default_source_file= default_source_file, default_ontology_backend= default_ontology_backend)
             
    def load_knowledge_framework(self, name=None, source= None, source_directory= None, backend= None):
        """Gets and initilises the ontology from the specified source.
        
        Parameters:
        - name: the name to be given to the knowledge framework. If None (default) the file name will be used.
        - source: filename to the ontology source. If None(default) the default ontology is used.
        - source_directory: where the system should look for ontology definition files. If None, the `GeologicalKnowledgeFramework` will decide.
        - backend: the ontology backend to be used. If None, the `GeologicalKnowledgeFramework` will decide."""
        source = source if source is not None else self.default_source_file
        name = name if name is not None else os.path.basename(source).split(os.path.extsep)[0]
        self.knowledge_framework_dict[name] = GeologicalKnowledgeFramework(name= name, source= source, source_directory= source_directory, backend= backend)
    
    def get_knowledge_framework(self,name= "default"):
        """Accessor to knowledge frameworks."""
        name = self.default if name == "default" else name
        assert len(self.knowledge_framework_dict) > 0, "No ontology has been loaded yet. Please use GeologicalKnowledgeManager().load_knowledge_framework() first"
        assert name in self.knowledge_framework_dict.keys(), "The specified ontology hasn't been loaded: "+name+\
            "\navailable ontology names are: "+"\n".join(self.knowledge_framework_dict.keys())
        return self.knowledge_framework_dict[name]
    
class GeologicalKnowledgeFramework(object):
    """A GeologicalKnowledgeFramework holds the definition of concepts and relationships describing knowledge.
    
    This is typically an overlay around a formal ontology definition, which also brings additional capabilities,
    such as algorithms and factories to achieve specific tasks and create objects.
    
    This knowledge framework also holds a registry of available constructors for making new objects:
    - registered_constructors: a dictionnary holding the constructors for a given class, sorted in order of preference,
         structured as {class_object or key: [constructor1, constructor2,...]}
    - constructor_conditions: conditions for the application of a constructor,
       e.g., regarding InterpretationSituation, interpretor status, and parameters.
        Structured as dictionnaries {constructor: function_to_evaluate_conditions}"""

    # registry of object constructors and conditions
    registered_constructors = {}
    constructor_conditions = {}
    
    def __init__(self, name, source, source_directory= None, backend= None):
        """Initialise a KnowledgeFramework form a given ontology file (source).
        
        Parameters:
        - name: should be the name under which this KnowledgeFramework is known in the manager
        - source: the source file for the ontology definition
        - source_directory: the directory where the source files for the ontology definition are looked for.
        If None (default) the default path provided by the `KnowledgeManager` is used.
        - backend: the ontology backend to be used for this knwoledge framework.
        If None (default) the default ontology backend provided by the `KnowledgeManager` is used."""
        self.name= name
        
        self._source_directory= None
        self.init_source_directory(source_directory)
        self.initialise_ontology_backend(backend)
        
        self.load_ontology(source)
            
    
    def init_source_directory(self, source_directory):
        """Initialises the folder where source files are searched.
        
        Parameters:
        - source_directory: if None, the previous value is used if it wasn't None, else the `GeologicalKnowledgeManager`default is used."""
        if source_directory is not None:
            self._source_directory= source_directory
        elif self._source_directory is None:
            self._source_directory= GeologicalKnowledgeManager().default_source_directory
    
    def initialise_ontology_backend(self, backend_name:str= None):
        """Initializes the ontology package used as a backend to access ontologies.
        
        This will:
        - try to import the backend as onto
        - set the default path for ontologies"""
                
        self._ontology_backend = None
        backend_name= GeologicalKnowledgeManager().default_ontology_backend if backend_name is None else backend_name
        if backend_name == "owlready2":
            try:
                import owlready2 as owl2 
                self._ontology_backend = owl2
                if self._source_directory not in self._ontology_backend.onto_path:
                    self._ontology_backend.onto_path.append(self._source_directory)
            except ImportError:
                raise ImportError("Your are trying to use Owlready2 as a backend for ontology management, but it doesn't appear to be installed."\
                "This is either because OwlReady2 is given as default option or because you asked for it."\
                "Please install the OwlReady2 package from https://owlready2.readthedocs.io"\
                "or give another backend through GeologicalKnowledgeManager().initialise_ontology_backend()")
                
            # also test if java is correctly installed & accessible, as it is used by owlready2 for reasoning
            try:
                os.system("java -version")
            except:
                raise ImportError("Java doesn't appear to be installed properly as the command `java -version` returned an error."\
                    "This error occured while loading owlready2 package as an ontology backend, because java is used for the reasoning engine.")
        else:
            raise Exception("The specified backed for ontology is not supported: "+backend_name)
          
        
    def load_ontology(self, source):
        """Loads the ontology specified by source.
        
        Parameters:
        - source: the source file for the ontology definition
        - source_directory: the directory where the source files for the ontology definition are looked for.
        If None (default) the default path provided by the `KnowledgeManager` is used."""
        self._source= source
        try:
            self._onto = self._ontology_backend.get_ontology(self._source).load()
        except Exception as err:
            raise Exception("Unexpected exception received while loading ontology:\n - source: {}\n - onto_path: {}".format(self._source, self._ontology_backend.onto_path))
        
    def __call__(self):
        return self._onto
        
    def get_ontology_backend(self):
        """Gets the ontology backend"""
        assert self._ontology_backend is not None, "Trying to access the ontology backend without initialising it."
        return self._ontology_backend
    
    def search(self, name= None, type= None, qualities= None, prepend_star=True) -> list:
        """Search function to interface the serach capabilities of the internal ontology
        
        Parameters:
        - name: the name of the search object (you can use * to replace any set of characters and ? to replace any single character)
        Note: if `prepend_star` a * is always prepended to allows the search to work because of the internal prefix names
        - type: the type of the searched observations as defined by the internal ontology
        - qualities: qualities to filter the observations.
        If `qualities` is a :
         * `str`: a single quality will be searched for with any value ("*"),
         * `list`: a list of qualities will be searched for with any values ("*")
         * `dict`: a list of qualities defined by the keys and with the associated values will be searched for"""
        if name is None:
            name = "*"
        elif prepend_star:
            name = "*"+name
        
        if isinstance(qualities,list):
            kargs = {quality_i: "*" for quality_i in qualities} 
        elif isinstance(qualities,str):
            kargs = {qualities: "*"}
        elif isinstance(qualities,dict):
            kargs = qualities
        else:
            kargs = {}
            assert (qualities is None) or isinstance(qualities,dict), "qualities should be given as either None, a str, a list, or a dict"
        if type is not None: kargs["type"] = type
        return self._onto.search(iri= name, **kargs)
    
    def format_qualities(self, **qualities):
        """Formats the qualities for setting an instance

        It will :
         - remove qualities that are not defined in the ontology
         - transform scalar values into vectors (for non functional properties)
         - filter out None values
        """
        
        formated_qualities = {key:val for key, val in qualities.items() 
                 if (val is not None) and (getattr(self._onto,key) is not None)
                 }
        for key, val in formated_qualities.items():
            if not getattr(self._onto,key).is_functional_for(None):
                formated_qualities[key] = val if isinstance(val,list) else [val]
        return formated_qualities

    def create_instance(self, class_type, name = None, physical_space= None, **qualities):
        """Creates an instance in the ontology
        
        Parameters:
        - class_type: the type of the class of the object to be created (as in the ontology, e.g., mogi().Surface).
        Note: the given class_type must be in the internal ontology stored in this manager.
        - name: a string giving the name of the object. If None (default) this name is set automatically
        - physical_space: if defined, a PhysicalSpaceRepresentation that handles the spatial qualities given in qualities
        - qualities: a series of possible qualities to be added to the object"""
        if class_type not in self._onto.classes():
            raise MalissiaBaseError("The proposed class type ({}) does not belong to this ontology ({})".format(class_type, self._onto))
            
        # checking name
        if name == "":
            logging.warning("Trying to create a '{}' with a blank name, passing None instead.".format(class_type))
            name = None
        if not isinstance(name,str):
            logging.warning("Trying to create a '{}' with a name that is not a string, using default naming instead.".format(class_type))
            name = None

        #reformating the qualities
        
        formated_qualities = self.format_qualities(**qualities)
        if physical_space:
            non_coordinate_qualities = physical_space.filter_qualities(**formated_qualities)
            new_individual = class_type(name= name, **non_coordinate_qualities)
            physical_space.set_object_coordinates(new_individual, **qualities)
        else:
            new_individual = class_type(name= name, **formated_qualities)

        return new_individual

    def remove_instance(self, instance):
        self._ontology_backend.destroy_entity(instance)
        
    def remove_all_instances(self, instances = None):
        if instances is None:
            instances = self._onto.individuals()
        for instance in instances:
            self.remove_instance(instance)
        
    def get_all_classes_of_instance(self,instance):
        """Returns all the classes of this instance and all its ancestors
        
        Parameters:
        - instance: an instance to be investigated
        Returns:
        - a set containing all the classes this instance belongs to including the mother classes
        Note: be carefull because this is returnin owl.Thing too and may cause issues if not handled
        """
        instance_classes = set(instance.is_instance_of)
        instance_classes.discard(self._ontology_backend.Thing)
        return set.union(*[instance_class.ancestors() for instance_class in instance_classes])
        
    def show_instance_qualities(self,instance):
        if instance is None: 
            print("The instance is None.")
            return
        print("instance:",instance)
        print("types:", ",".join([str(i) for i in instance.is_a]))
        print("properties:")
        for prop in instance.get_properties():
            print("|- {}:{}".format(prop.name, prop[instance]))
            
    def get_all_instances(self):
        return list(self._onto.individuals())
    
    def show_all_instance_qualities(self, instances= None):
        instances = instances if instances is not None else self.get_all_instances()
        print("Number of instances:",len(instances))
        for i in instances:
            self.show_instance_qualities(i)
            
    def has_quality(self, object, quality):
        return getattr(self._onto, quality) in object.get_properties()
    
    def has_qualities(self, object, qualities):
        qualities = np.array(qualities).ravel()
        return np.all([self.has_quality(object, quality_i) for quality_i in qualities])
            
    def get_possible_interpretations_of(self, object):
        """Returns a list of classes potentially explaining this object"""
        object_classes = object.is_instance_of
        if len(object_classes) == 0:
            raise MalissiaBaseError("Trying to interprete an object without class.")
        possible_interpretations = [interpretation for class_i in object_classes 
                                    for interpretation in class_i.has_Possible_Explanation ] 
        return possible_interpretations
        
    def get_objects_potentially_explained_by(self, object):
        """Returns the classes this object is potentially explaining."""
        return object.is_Possible_Explanation_Of
    
    def isinstance(self, candidate_object, candidate_class):
        """Checks whether an object is an instance of a given class"""
        return isinstance(candidate_object, candidate_class)
    
    def issubclass(self, candidate_subclass, candidate_class):
        """Checks whether an object is an instance of a given class"""
        return issubclass(candidate_subclass, candidate_class)
    
    def has_representation_of_type(self, candidate_object, representation_type, return_representations= False):
        """Checks if an instance has a representation of a given type"""
        candidate_object
        if (candidate_object.has_Representation is None) or (len(candidate_object.has_Representation) < 1):
            return False
        reps = np.array(candidate_object.has_Representation)
        selected_representations = [self.isinstance(rep_i, representation_type) for rep_i in reps]
        if return_representations:
            return reps[selected_representations]
        else:
            return np.any(selected_representations)
        
    def has_point_representation(self, candidate_object, return_representations= False):
        return self.has_representation_of_type(candidate_object, self._onto.Point, return_representations)
    
    @classmethod
    def register_constructor(cls, object_class_name, constructor, condition = None):
        """Registers a constructor for the given ontology class
        
        Parameters:
        - object_class_name: the type of object to be created (typically, the ontology class name)
        - constructor: a function that can be called to generate the given object class.
          It will typically be method of the GeologicalKnowledgeFramework and called by
          constructor( knowledge_framework= None, interpretation_situation = None, interpretation_status = None, **kargs)
        """
        if (object_class_name is None) or (not isinstance(object_class_name, str)):
            raise MalissiaBaseError("Registering constructor for an undefined object class.")
        if constructor is None: raise MalissiaBaseError("Registering  an undefined constructor for an object class.")
        
        if object_class_name in cls.registered_constructors:
            cls.registered_constructors[object_class_name] += [constructor]
        else:
            cls.registered_constructors[object_class_name] = [constructor]
        cls.constructor_conditions[constructor] = condition
            
    def filter_explainable_features(self, object_class, features):
        """Selects the features that can be explained by a given ontology class.
        
        Parameters:
        - object_class [object]: a type of object used as an explanation
        - features: the list of features to be filtered.
        Returns:
        - a list of features that can be explained by the proposed class"""
        explainable_class_set = set(self.get_objects_potentially_explained_by(object_class))
        return [feature_i for feature_i in features 
                if not self.get_all_classes_of_instance(feature_i).isdisjoint(explainable_class_set)]
                # test wether the is an intersection between the classes of the instance and the explainable classes
        
    def select_object_constructor(self, object_class_name, 
                                    interpretation_situation = None,
                                    interpretation_status = None,
                                    random_choice = False,
                                    debug= False,
                                    **kargs):
        """Factory function that generates a constructor for the given ontology class
        
        Parameters:
        - object_class_name: the type of object to be created (typically, the ontology class name)
        - interpretation_situation: a `InterpretationSituation` providing a context for the creation
        - interpretation_status: the status of the current interpretation process,
        which might provide contextual information to decide which constructor to generate
        - random_choice: if False (default), the first available constructor is selected, else it is picked randomly
        - debug: if True, some debug information is sent, default False. 
        - kargs: keyword arguments passing available qualities and properties.
        This will be used to check whether required information is available.
        
        Return:
        - a function that can be called for creating a new instance of the given object.
        """
        if debug:
            print("Looking for a constructor for:", object_class_name)
            print(" - registered constructors:\n", "\n".join([str(constructor) for constructor in self.registered_constructors[object_class_name]]))
        if object_class_name not in self.registered_constructors:
            raise MalissiaBaseError("No available constructor for {}.".format(object_class_name))
        candidate_constructors = [constructor for constructor in self.registered_constructors[object_class_name]
                                  if self.constructor_conditions[constructor](
                                        knowledge_framework = self,
                                        interpretation_situation = interpretation_situation,
                                        interpretation_status = interpretation_status,
                                        **kargs)
                                  ]
        if debug:
            print(" - candidate constructors:\n", "\n".join([str(constructor) for constructor in candidate_constructors]))
        if len(candidate_constructors) == 0:
            raise MalissiaBaseError("No available constructor for {} in these conditions.".format(object_class_name))
        
        selected_constructor = random.choice(candidate_constructors) if random_choice else candidate_constructors[0]
        if debug: print("Selected constructor:",str(selected_constructor))
        return selected_constructor
        
    def sync_reasoner(self, **kargs):
        """Synchronise the reasoner.
        
        Parameters:
        - **kargs:
        |-infer_property_values"""
        self._ontology_backend.sync_reasoner(**kargs)
        
    def __str__(self):
        """Describe the knowledge framework"""
        desc = ["Geological Knowledge Framework:"]
        desc += [" |- Name: {}".format(self.name)]
        desc += [" |- Backend: {}".format(self._ontology_backend)]
        desc += [" |- Source: {}".format(self._source)]
        desc += [" |- Ontology: {}".format(self._onto)]
        return "\n".join(desc)
    

### Constructors

#### Point

In [ ]:

def format_coord_dicts(coord_labels, coords):
    label_keys = ["coord{}_label".format(i+1) for i in range(len(coord_labels))]
    coord_keys = ["coord{}".format(i+1) for i in range(len(coords))]
    coord_label_dict = {label_key:[coord_label_i] for label_key,coord_label_i in zip(label_keys, coord_labels)}
    coord_dict = {coord_key:[coord_i] for coord_key,coord_i in zip(coord_keys, coords)}
    return coord_label_dict, coord_dict

def constructor_point(knowledge_framework, coord_labels, coords, name= None):
    coord_label_dict, coord_dict = format_coord_dicts(coord_labels=coord_labels, coords= coords)
    point =knowledge_framework().Point(name= name, **coord_label_dict, **coord_dict)
    
    # it is represented by himself
    point.has_Representation = [point]
    return point

##### Point Anomalies

#### Vector

In [ ]:
def constructor_vector(knowledge_framework, coord_labels, coords, name= None):
    coord_label_dict, coord_dict = format_coord_dicts(coord_labels= coord_labels, coords= coords)
    vector =  knowledge_framework().Vector(name= name, **coord_label_dict, **coord_dict)
    
    # it is represented by himself
    vector.has_Representation = [vector]
    return vector

##### Vector Anomalies

#### Planar_Surface

In [ ]:
def get_coord_labels(plan):
    node = plan.has_Node0[0]
    return [node.coord1_label[0],node.coord2_label[0],node.coord3_label[0]]

def get_nodes(plan):
    return [plan.has_Node0[0], plan.has_Node1[0], plan.has_Node2[0], plan.has_Node3[0]]

def get_coord(nodes):
    """returns the coordinates of a series of nodes as an array (node_index, coord_index)"""
    return [[node_i.coord1[0],node_i.coord2[0],node_i.coord3[0]] for node_i in nodes]
    
def compute_center(nodes):
    coords = get_coord(nodes= nodes)
    center_coord = np.mean(coords,axis=0).tolist() 
    return center_coord

def set_center(knowledge_framework, plan):
    nodes = get_nodes(plan= plan)
    center_coord = compute_center(nodes= nodes) 
    coord_labels = get_coord_labels(plan= plan)
    
    name = plan.name + "_center"
    center = constructor_point(knowledge_framework= knowledge_framework, coord_labels= coord_labels, coords= center_coord, name= name)
    plan.has_Center = [center]
    return center

def compute_principal_vectors_from_nodes(nodes):
    coords = np.array(get_coord(nodes= nodes))
    u = coords[1] - coords[0]
    v = coords[3] - coords[0]
    normal = np.cross(u,v)
    normal = normal/np.linalg.norm(normal)
    return u.tolist(), v.tolist(), normal.tolist()

def compute_size_from_nodes(nodes):
    coords = np.array(get_coord(nodes= nodes))
    u = coords[1] - coords[0]
    size = np.linalg.norm(u)
    return float(size)

def compute_dip_dir_from_normal(normal):
    x,y,z = np.sign(normal[2]) * np.array(normal) / np.linalg.norm(normal)
    if z == 1:
        return 0.,0.
    dip = np.rad2deg(np.arccos(z))
    dip_dir = np.rad2deg(np.arctan2(x,y)) % 360
    return float(np.round(dip, 3)), float(np.round(dip_dir, 3))

def set_attitude(knowledge_framework, plan):
    nodes = get_nodes(plan= plan)
    coord_labels = get_coord_labels(plan= plan)
    u, v, normal = compute_principal_vectors_from_nodes(nodes= nodes)
    size = compute_size_from_nodes(nodes= nodes)
    plan.size = [size]
    
    normal_name = plan.name + "_normal"
    normal_entity = constructor_vector(knowledge_framework= knowledge_framework, coord_labels= coord_labels, coords= normal, name= normal_name)
    plan.has_Normal = [normal_entity]
    
    dip, dip_dir = compute_dip_dir_from_normal(normal= normal)
    plan.dip = [dip]
    plan.dip_dir = [dip_dir]
    return dip, dip_dir, normal, size

def set_nodes(plan, nodes):
    if(len(nodes) != 4): raise MalissiaBaseError("Please give 4 nodes in, {} were given.".format(len(nodes)))
    plan.has_Node0 = [nodes[0]]
    plan.has_Node1 = [nodes[1]]
    plan.has_Node2 = [nodes[2]]
    plan.has_Node3 = [nodes[3]]
    plan.has_Representation = nodes
    
def create_planar_surface(knowledge_framework, nodes, name= None):
    plan = knowledge_framework().Planar_Surface(name= name)
    set_nodes(plan,nodes)
    return plan

def constructor_planar_surface_from_nodes(knowledge_framework, nodes, name= None):
    plan = create_planar_surface(knowledge_framework= knowledge_framework, nodes= nodes, name= name)
    center = set_center(knowledge_framework= knowledge_framework, plan= plan)
    dip, dip_dir, normal, size = set_attitude(knowledge_framework= knowledge_framework, plan= plan)
    return plan

def create_nodes_from_coords(knowledge_framework, coords, coord_labels, name= None):
    names = [None if name is None else name + "_N" + str(i) for i in range(len(coords))]
    nodes = [
        constructor_point(knowledge_framework= knowledge_framework, coord_labels= coord_labels, coords= coord_i, name= name_i)
        for coord_i, name_i in zip(coords,names)
    ]
    return nodes

def constructor_planar_surface_from_coords(knowledge_framework, coords, coord_labels, name= None):
    nodes = create_nodes_from_coords(knowledge_framework= knowledge_framework, coords= coords, coord_labels= coord_labels, name= name)
    return constructor_planar_surface_from_nodes(knowledge_framework= knowledge_framework, nodes= nodes)

def compute_normal_from_dip_dir(dip, dip_dir, polarity= 1):
    dip_rad = np.deg2rad(dip)
    dip_dir_rad = np.deg2rad(dip_dir)
    z = np.cos(dip_rad)
    h = np.sin(dip_rad)
    y = h * np.cos(dip_dir_rad)
    x = h * np.sin(dip_dir_rad)
    normal = polarity * np.array([x,y,z])
    return normal.tolist()

def compute_principal_vectors_from_dip_dir(dip, dip_dir, polarity= 1):
    dip_rad = np.deg2rad(dip)
    dip_dir_rad = np.deg2rad(dip_dir)
    nz =  np.cos(dip_rad)
    h = np.sin(dip_rad)
    hy = np.cos(dip_dir_rad)
    hx = np.sin(dip_dir_rad)
    ny = h * hy
    nx = h * hx
    normal = polarity * np.array([nx,ny,nz])
    
    uz = -h
    ux = nz * hx
    uy = nz * hy
    dip_vector = np.array([ux,uy,uz])
    
    az_vector = np.cross(normal, dip_vector)
    
    return normal.tolist(), dip_vector.tolist(), az_vector.tolist()

def compute_coords(center, dip_vector, az_vector, size):
    center     = np.array(center)
    dip_vector = np.array(dip_vector)
    az_vector  = np.array(az_vector)
    coords = np.array([
        center - size/2 * dip_vector - size/2 * az_vector,
        center + size/2 * dip_vector - size/2 * az_vector,
        center + size/2 * dip_vector + size/2 * az_vector,
        center - size/2 * dip_vector + size/2 * az_vector
    ])
    return coords.tolist()

def constructor_planar_surface_from_center_attitude(knowledge_framework, coord_labels, center, dip, dip_dir, size= None, polarity= True, name= None):
    normal, dip_vector, az_vector = compute_principal_vectors_from_dip_dir(dip= dip, dip_dir= dip_dir, polarity= polarity)
    coords = compute_coords(center= center, dip_vector= dip_vector, az_vector= az_vector, size= size)
    nodes = create_nodes_from_coords(knowledge_framework= knowledge_framework, coords= coords, coord_labels= coord_labels)
    plan = create_planar_surface(knowledge_framework= knowledge_framework, nodes= nodes, name= name)
    
    plan.dip = [float(dip)]
    plan.dip_dir = [float(dip_dir)]
    plan.polarity = [translate_polarity(polarity)]
    plan.size = [float(size)]
    
    center_name = plan.name + "_center"
    center_entity = constructor_point(knowledge_framework= knowledge_framework, coord_labels= coord_labels, coords= center, name= center_name)
    plan.has_Center= [center_entity]
    
    normal_name = plan.name + "_normal"
    normal_entity = constructor_vector(knowledge_framework= knowledge_framework, coord_labels= coord_labels, coords= normal, name= normal_name)
    plan.has_Normal = [normal_entity]
    return plan
    

##### Planar Surface anomalies

#### Observations

In [ ]:
def conditions_for_constructor_observation(knowledge_framework:GeologicalKnowledgeFramework, **kargs):
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for observation constructor: knowledge framework")
        return False
    if ("dataset" not in kargs) or (kargs["dataset"] is None):
        print("Wrong condition for observation constructor: dataset")
        return False
    dataset = kargs["dataset"]
    if (dataset.physical_space is None):
        print("Wrong condition for observation constructor: physical space")
        return False
    if not np.all([coord in kargs for coord in dataset.physical_space.coordinate_labels]):
        print("Wrong condition for observation constructor: coord")
        return False
    return True
    
def constructor_observation(knowledge_framework= None, name: str= None,
                            dataset= None, physical_space= None, coord_labels= None,
                            **kargs):
    """Constructor of observations
    
    Parameters:
    - knowledge_framework: defining the existing objects, if None it is inferred from the dataset if given
    - name: the name of the observation (similar to an observation id). If None a default one is given.
    - dataset: (optional) the dataset to which this observation belongs.
    If given, the physical space is taken from there unless it is specified
    - physical_space: (optional) the physical space in which this observation is defined.
    If not given, it is taken from the dataset, or if not dataset is provided, the coordinate_labels must be given
    - coord_labels: (optional) a list of coordinate names, required only if no physical space is provided
    - kargs: keyword arguments, containing:
        - spatial coordinates with names matching the dataset.physical_space coordinate labels
        - any other quality that should be associated with the observation
         * size: should be set, this is the size of the area represented by this observation
         * dip and dip_dir: optional, make it an orientation observation, should both be set together
         * polarity: optional, only used if dip an ddip_dir are set, boolean indicating the polarity of the observed feature.
         True means up (or towards the dip_dir if vertical), False is the opposite
         * occurrence: optional make it an occurrence observation, True (thing has been observed)
         or False (it has been observed that the thing is not here). Note: if the observation is not made on the
         occurrence of the observed object, don't set this property, do not set it to False.
        - qualities whose values is None are not set
        - Note that the dataset.physical_space will be asked to filter the provided information
        
    Return:
    - the created observation
    """
    
    if knowledge_framework is None:
        if dataset is not None:
            knowledge_framework = dataset.knowledge_framework
        else:
            raise MalissiaBaseError("provide at least a knowledge framework or a dataset")
    
    # create Observation
    obs = knowledge_framework().PointBased_Observation(name= name)

    # create a point as geometrical support
    physical_space = physical_space if physical_space is not None else \
        (dataset.physical_space if dataset is not None else None)
    coord_labels = physical_space.coordinate_labels if physical_space is not None else coord_labels
    if coord_labels is None:
        raise MalissiaBaseError("coordinate labels must be specified somehow.")
    coords = [kargs[i] for i in coord_labels]
    point = constructor_point(knowledge_framework= knowledge_framework,
                              coord_labels= coord_labels, coords= coords,
                              name= obs.name + "_point")
    obs.has_Center = [point]
    obs.has_Representation = [point]
    
    # add size
    if "size" in kargs:
        obs.size = [kargs["size"]]
    elif physical_space is not None:
        domain_size = float(max(physical_space.get_size()))
        obs.size = [domain_size / 10]
    else:
        obs.size = [1]
    
    # add observed qualities
    if "occurrence" in kargs:
        obs.occurrence = [kargs["occurrence"]]
    if ("dip" in kargs) and ("dip_dir" in kargs):
        obs.dip     = [kargs["dip"]]
        obs.dip_dir = [kargs["dip_dir"]]
        if "polarity" in kargs:
            obs.polarity = [translate_polarity(kargs["polarity"])]
        else:
            obs.polarity = [True]
            
    # todo : find or create observed object
    
    return obs

if "Observation" in GeologicalKnowledgeFramework.registered_constructors:
    del GeologicalKnowledgeFramework.registered_constructors["Observation"]
GeologicalKnowledgeFramework.register_constructor("Observation", 
                                                  constructor= constructor_observation,
                                                  condition= conditions_for_constructor_observation)

##### Observation anomalies

In [ ]:
# internal consistency anomaly
# eg. dip, dip_dir out of range
# must have at least (dip and dip_dir) or occurrence

#### Situation

In [ ]:
class InterpretationSituation(object):
    """Defines a situation of interpretation.
    
    A situation is gathering:
    - self.features: features to be explained
    - self.context: an interpretation context, ie. elements to be considered during the interpretation
    - self.process: the interpretation process in which this situation arises"""
    
    def __init__(self, features, process= None, context = None, knowledge_framework= None) -> None:
        """Initialises the situation
        
        Parameters:
         - self.features: features to be explained
         - self.context: an interpretation context, ie. elements to be considered during the interpretation
         - self.process: the interpretation process in which this situation arises
         """
        self.features = np.array(features).ravel()
        self.context = np.array(context).ravel() if context is not None else None
        self.candidate_explaining_object = None
        self.process = process
        self.knowledge_framework = knowledge_framework if knowledge_framework is not None else \
            (process.knowledge_framework if process is not None else None)
      

#### Stratigraphy

In [ ]:
def conditions_for_constructor_surface_part_from_interpretation(
                                            knowledge_framework:GeologicalKnowledgeFramework,
                                            interpretation_situation:InterpretationSituation, **kargs):
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for Surface Part constructor: knowledge framework")
        return False
    if (interpretation_situation is None):
        print("Wrong condition for Surface Part constructor: interpretation situation")
        return False
    if  (interpretation_situation.process is None) or (interpretation_situation.process.physical_space):
        print("Wrong condition for Surface Part constructor: physical space")
        return False
    if  (interpretation_situation.features is None) or (len(interpretation_situation.features) == 0):
        print("Wrong condition for Surface Part constructor: features")
        return False
    
    onto_class = knowledge_framework().Stratigraphic_Part
    explainable = knowledge_framework.get_objects_potentially_explained_by(onto_class)
    if not np.any([feature in explainable for feature in interpretation_situation.features]):
        print("Wrong condition for Surface Part constructor: no explainable feature in situation")
        return False
    return True
    
def get_physical_space_in_constructor(interpretation_situation, physical_space= None, **kargs):
    """Gets a physical representation space from an interpreted situation of context"""
    if physical_space is not None:
        return physical_space
    if interpretation_situation is not None \
    and interpretation_situation.process is not None \
    and interpretation_situation.process.physical_space is not None:
        return interpretation_situation.process.physical_space
    else:
        raise MalissiaBaseError("Wrong conditions for physical space definition.")
    
def location_constructor(features, physical_space, method= "average", return_as_dict= False):
    """Estimated a location from a given situation and or space
    
    Parameters:
    - features: a list of features that can inform the location, if empty the location is set from the physical_space
    - physical_space: the space in whichthe new location is to be defined
    - method: determines how th e new location should be estimated (pick, random, average)
    Returns:
    - a dict matching each coordinate label with its coordinate value
    """ 
    if physical_space is None: raise MalissiaBaseError("a physical space is required, use get_physical_space_in_constructor")
    if len(features) == 0:
        # set from the physical space
        if method == "average":
            location = physical_space.get_center()
        else:
            location = physical_space.generate_random_location()
    else:
        coords = np.array([physical_space.get_object_coordinates(feature_i) for feature_i in features])
        if method == "average":
            # take the average position as new position
            location = np.mean(coords, axis= 0)
        elif method == "pick":
            # pick a random location from the dataset
            location = np.random.default_rng().choice(coords)
        elif method == "random":
            # generate a random location in the space covered by the data points
            mean = np.mean(coords, axis= 0)
            cov = np.cov(coords, rowvar=False)
            location = np.random.default_rng().multivariate_normal(mean, cov)
        return physical_space.coordinates_to_dict(location) if return_as_dict else location.tolist()
        
def translate_polarity(polarity):
    return bool(polarity > 0)

def format_attitude_qualities(dip, dip_dir, polarity= True, size= 1):
    return {"dip":float(dip),"dip_dir":float(dip_dir), "polarity": translate_polarity(polarity), "size": float(size)}

def attitude_constructor_from_space(physical_space):
    """Estimated attitude from a list of points"""
    dip = physical_space.generate_random_dip()
    dip_dir = physical_space.generate_random_dip_dir()
    polarity = physical_space.generate_random_polarity()
    size = physical_space.generate_random_size()
    return format_attitude_qualities(dip= dip, dip_dir= dip_dir, polarity= polarity, size= size)
    
def attitude_constructor_from_attitude_features(features, physical_space, method= "average"):
    """Estimated attitude from a list of features having attitudes"""
    if method == "pick":
        picked_feature = np.random.default_rng().choice(features)
        dip = picked_feature.dip[0]
        dip_dir = picked_feature.dip_dir[0]
        polarity = picked_feature.polarity[0]
        size = picked_feature.size[0]
    elif method == "average":
        dip = [feature_i.dip[0] for feature_i in features]
        dip_dir = [feature_i.dip_dir[0] for feature_i in features]
        polarities = [(1 if feature_i.polarity[0] else -1) for feature_i in features]
        vectors = [compute_normal_from_dip_dir(dip_i, dip_dir_i, polarity_i) for dip_i, dip_dir_i, polarity_i in zip(dip, dip_dir, polarities) ]
        normal = physical_space.compute_average_vector(vectors)
        dip, dip_dir, polarity = physical_space.compute_dip_dir_from_normal(normal)
        size = np.mean([feature_i.size[0] for feature_i in features])
        polarity = physical_space.generate_random_polarity()
    else:
        raise MalissiaBaseError("unimplemented method: "+method)
    return format_attitude_qualities(dip= dip, dip_dir= dip_dir, polarity= polarity, size= size)

def attitude_constructor_from_points(points, physical_space, method= "average"):
    """Estimated attitude from a list of points"""
    if len(points) < 2:
        raise MalissiaBaseError("not enough points to estimate attitude")
    if len(points) == 2:
        line = physical_space.compute_line_attitude_from_two_points(*points)
        if method == "average":
            plane = line
        else:
            raise MalissiaBaseError("method not implemented yet: " + method)
    else:
        if method == "average":
            plane = physical_space.compute_attitude_from_points(points)
        else:
            raise MalissiaBaseError("method not implemented yet: " + method)
        
    polarity = physical_space.generate_random_polarity()
    return format_attitude_qualities(dip= plane["dip"], dip_dir= plane["dip_dir"], size= plane["size"], polarity= polarity)

def attitude_constructor_from_single_feature(feature, knowledge_framework= None):
    # if it has dip and dip_dir -> use it
    dip = feature.dip[0] if knowledge_framework.has_quality(feature, "dip") else physical_space.generate_random_dip()
    dip_dir = feature.dip_dir[0] if knowledge_framework.has_quality(feature, "dip_dir") else physical_space.generate_random_dip_dir()
    return format_attitude_qualities(dip= dip, dip_dir= dip_dir)
    
def attitude_constructor(features, physical_space, knowledge_framework= None, method= "average"):
    """Estimated the attitude of a feature from a given situation and or space
    
    Parameters:
    - features: a list of features that can inform the location
    - physical_space: the space in whichthe new location is to be defined
    - method: determines how the new attitude should be estimated (pick, random, average)""" 
    if physical_space is None: raise MalissiaBaseError("a physical space is required, use get_physical_space_in_constructor")
    knowledge_framework = GeologicalKnowledgeManager().get_knowledge_framework() if knowledge_framework is None else knowledge_framework
    
    if len(features) == 0:
        return attitude_constructor_from_space(physical_space)
    elif len(features) == 1:
        return attitude_constructor_from_single_feature(features[0], knowledge_framework= knowledge_framework)
    else:
        # filtering point features
        features = np.array(features)
        is_instance_of_point = [knowledge_framework.has_point_representation(feature_i) for feature_i in features]
        point_features = features[is_instance_of_point]
        points = [physical_space.get_object_coordinates(point_feature_i) for point_feature_i in point_features]
        
        is_attitude_instance = [knowledge_framework.has_qualities(feature_i,["dip","dip_dir"]) for feature_i in features]
        attitude_features = features[is_attitude_instance]
        
        # if only points then create from them
        if np.all(is_attitude_instance):
            return attitude_constructor_from_attitude_features(attitude_features, physical_space= physical_space, method= method)
        elif np.all(is_instance_of_point):
            return attitude_constructor_from_points(points, physical_space= physical_space, method= method)
        else:
            print("Warning: surfaces are ignored while constructing from both points and surfaces. To be implemented.")
            return attitude_constructor_from_points(points, physical_space= physical_space, method= method)
    
def constructor_surface_part_from_interpretation(knowledge_framework, interpretation_situation,
                                                 physical_space= None, name= None, **kargs):
    """Constructor of observations
    
    Parameters:
    - knowledge_framework: the knowledge framework in which this object must be created
    - interpretation_situation: the interpretation situation selected in the given state of interpretation process
    - physical_space: the physical representation space where the surface is to be defined, if None the one from the interpretation_situation.process is used
        
    Return:
    - the created Surface Part
    """
    
    physical_space = get_physical_space_in_constructor(interpretation_situation, physical_space, **kargs)
    
    constructed_class = knowledge_framework().Stratigraphic_Part
    explainable_features = knowledge_framework.filter_explainable_features(constructed_class, interpretation_situation.features)
    if len(explainable_features) == 0:
        print("Warning: No explainable feature in the situation with this interpretation. To be implemented.")
        return None
    
    #estimate location
    location_qualities = location_constructor(features= explainable_features, physical_space= physical_space, method= "average")
    
    # estimate attitude
    attitude_qualities = attitude_constructor(features= explainable_features, physical_space= physical_space,
                                              knowledge_framework= knowledge_framework, method= "average")
    
    # estimate size
    size = attitude_qualities["size"]
    
    # Instanciate a PlanarSurface
    name_rep = None if name is None else name+"_rep"
    surface_representation = constructor_planar_surface_from_center_attitude(
                                    knowledge_framework= knowledge_framework,
                                    coord_labels= physical_space.coordinate_labels,
                                    center= location_qualities,
                                    dip= attitude_qualities["dip"],
                                    dip_dir= attitude_qualities["dip_dir"],
                                    polarity= attitude_qualities["polarity"],
                                    size= size, 
                                    name= name_rep
                                    )

    # Instanciate a Stratigraphic_Part pointing to the planarsurface representation
    class_type = knowledge_framework().Stratigraphic_Part
    surface_part = knowledge_framework.create_instance(class_type, name, has_Representation= surface_representation, **kargs)
    
    # explanation relationship
    surface_part.explain = explainable_features
     
    return surface_part

if "SurfacePart" in GeologicalKnowledgeFramework.registered_constructors:
    del GeologicalKnowledgeFramework.registered_constructors["SurfacePart"]
GeologicalKnowledgeFramework.register_constructor("SurfacePart", 
                                                  constructor= constructor_surface_part_from_interpretation,
                                                  condition= conditions_for_constructor_surface_part_from_interpretation)

##### Stratigraphic part anomalies

In [ ]:
# internal consistency anomaly

## DippingStratigraphyAnomaly
## anomaly for a StratigraphicPart with a planar geometry having a dip > threshold 

## DipVariationAnomaly
## anomaly for a StratigraphicParts with a ComplexSurface representation
## when two successive PlanarSurfaces have a variation of orientation

## DiscontinuousStratigaphyAnomaly
## anomaly for a StratigraphicParts with a ComplexSurface representation
## when two successive PlanarSurfaces do not intersect properly

## StratigraphyAnomaly
## Anomaly for a stratigraphic surface when its geometry
## doesn't reach the border of the domain

# ExplanationAnomaly
## an explained object geometry doesn't match the explaining object geometry
## distance between surface too big, orientation too different

### Anomalies

In [ ]:
class Anomaly:
    """A class for constructing anomalies"""
    def __init__(self, knowledge_framework = None):
        """Initialise base class attributes"""
        self.knowledge_framework = knowledge_framework if knowledge_framework is not None else GeologicalKnowledgeManager().get_knowledge_framework()
        pass


def dippingStratigraphyAnomaly_constructor(knowledge_framework:GeologicalKnowledgeFramework,
                                            related_object = []):
    """ constructor method for creating an instance of DippingStratigraphyAnomaly"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for DippingStratigraphyAnomaly_constructor : knowledge framework")
        return False
    anomaly = knowledge_framework().DippingStratigraphyAnomaly( is_Related_To = related_object)
    return anomaly

def dipVariationAnomaly_constructor(knowledge_framework:GeologicalKnowledgeFramework, 
                                    related_objects = []):
    """ constructor method for creating an instance of DipVariationAnomaly"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for DipVariationAnomaly_constructor : knowledge framework")
        return False
    anomaly = knowledge_framework().DipVariationAnomaly(is_Related_To = related_objects)
    return anomaly

def discontinuousStratigaphyAnomaly_constructor(knowledge_framework:GeologicalKnowledgeFramework,
                                                 discontinuous_objects = [], 
                                                 discontinuity_space = []):
    """ constructor method for creating an instance of DiscontinuousStratigaphyAnomaly"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for DiscontinuousStratigaphyAnomaly_constructor : knowledge framework")
        return False
    anomaly = knowledge_framework().DiscontinuousStratigaphyAnomaly(is_Related_To = discontinuous_objects,
                                                                 has_Occurrence_Space =  discontinuity_space )
    return anomaly

def stratigraphyAnomaly_constructor(knowledge_framework:GeologicalKnowledgeFramework, 
                                    stratigraphic_surface = [], occurence_spaces = []):
    """ constructor method for creating an instance of StratigraphyAnomaly"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for StratigraphyAnomaly_constructor : knowledge framework")
        return False
    anomaly = knowledge_framework().StratigraphyAnomaly(is_Related_To = stratigraphic_surface, 
                                                      has_Occurrence_Space = occurence_spaces)
    return anomaly


def explanationAnomaly_constructor(knowledge_framework:GeologicalKnowledgeFramework, 
                                   Explained_Object = [],
                                   Explaining_Object= []):
    """ constructor method for creating an instance of ExplanationAnomaly"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for ExplanationAnomaly_constructor : knowledge framework")
        return False
    anomaly = knowledge_framework().ExplanationAnomaly( is_Related_To_Explained_Object = Explained_Object,
                                                     is_Related_To_Explaining_Object = Explaining_Object )
    return anomaly


def limbsPolarityAnomaly_constructor(knowledge_framework:GeologicalKnowledgeFramework,
                                            related_limbs = []):
    """ constructor method for creating an instance of limbsPolarityAnomaly"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for DippingStratigraphyAnomaly_constructor : knowledge framework")
        return False
    anomaly = knowledge_framework().DippingStratigraphyAnomaly( is_Related_To = related_limbs)
    return anomaly


### to do define how to create the occuence space (through representational voxel ! )
## how to related by its physical qualities noodes coord


### Folds

In [ ]:
def stratigraphic_part_to_limb_constructor(knowledge_framework:GeologicalKnowledgeFramework, stratigraphic_part):
    """ constructor method for creating an instance of a limb based on an existing stratigraphic part"""
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        print("Wrong condition for DippingStratigraphyAnomaly_constructor : knowledge framework")
        return False
    return stratigraphic_part.is_a.append(knowledge_framework.Fold_Limb)   



def fold_instance_constructor(knowledge_framework:GeologicalKnowledgeFramework):
    """ constructor method for creating an instance of a Chevron_Fold"""
    return knowledge_framework.Chevron_Fold()


def fold_constructor(knowledge_framework:GeologicalKnowledgeFramework, 
                     Stratigraphic_Part1, Stratigraphic_Part2):
    if (knowledge_framework is None) or (knowledge_framework.name != "mogi"):
        raise ValueError ('Wrong condition for DippingStratigraphyAnomaly_constructor : knowledge framework')
    # creating an instance of a Chevron_Fold
    fold = knowledge_framework.Chevron_Fold()
    # creating an instance of limbs
    limb1 = stratigraphic_part_to_limb_constructor(knowledge_framework, Stratigraphic_Part1)
    limb2 = stratigraphic_part_to_limb_constructor(knowledge_framework, Stratigraphic_Part2)

    fold.has_Limb1 = limb1
    fold.has_Limb2 = limb2
    return fold    



    

In [ ]:
import numpy as np
import math
import random
from scipy.linalg import null_space
from sympy import Point, Point3D, Line3D, Plane, Polygon, Line, Float
import sympy as sp 
from scipy.spatial.transform import Rotation as R


def compute_fold_axis_non_oriented(limb1_normal, limb2_normal):
    limb1_normal = np.array(limb1_normal).astype(float)
    limb2_normal = np.array(limb2_normal).astype(float)
    fold_axis = np.cross(limb1_normal, limb2_normal)
    fold_axis_norm = np.linalg.norm(fold_axis)
    if fold_axis_norm == 0:
        raise MalissiaBaseError("Limbs are parallel.")
    return fold_axis / fold_axis_norm

def compute_axial_surface_normal(limb1_normal, limb2_normal, fold_axis):
    limb1_normal = np.array(limb1_normal).astype(float)
    limb2_normal = np.array(limb2_normal).astype(float)
    axial_surface_normal = np.cross(fold_axis, limb1_normal+limb2_normal)
    return axial_surface_normal / np.linalg.norm(axial_surface_normal)

def compute_direction_toward_hinge_along_limb_1(limb_normal, fold_axis):
    along_limb_vec = list(map(float,-1 * np.cross(limb_normal,fold_axis)))
    return along_limb_vec / np.linalg.norm(along_limb_vec)

def compute_direction_toward_hinge_along_limb_2(limb_normal, fold_axis):
    along_limb_vec = list(map(float, np.cross(limb_normal,fold_axis)))

    return along_limb_vec / np.linalg.norm(along_limb_vec)

def calculate_square_normal(vertex1, vertex2, vertex3, vertex4):
    """
    Calculate the normal vector of a square surface defined by its four vertices.
    """
    # Choose three non-collinear vertices to define a plane (e.g., vertices 1, 2, and 3)
    point1 = np.array(vertex1)
    point2 = np.array(vertex2)
    point3 = np.array(vertex3)

    # Calculate two vectors in the plane
    vector1 = point2 - point1
    vector2 = point3 - point1

    # Calculate the normal vector by taking the cross product of vector1 and vector2
    normal_vector = np.cross(vector1, vector2)
    
    # Optionally, normalize the normal vector to have a unit length
    magnitude = np.linalg.norm(normal_vector)
    normalized_vector = normal_vector / magnitude

    return normalized_vector

def compute_square_center(vertex1, vertex2, vertex3, vertex4):
    return np.array((vertex1 + vertex2 + vertex3 + vertex4) / 4)    

def compute_intersection_point_beta(limb_1_nodes , limb_2_nodes):

    # calculate n1, n2, c1, c2, v1, f_axe
    n1 = sp.Point(calculate_square_normal(*limb_1_nodes))
    n2 = sp.Point(calculate_square_normal(*limb_2_nodes))
    c1 = sp.Point(compute_square_center(*limb_1_nodes))
    c2 = sp.Point(compute_square_center(*limb_2_nodes))
    f_axe = sp.Point(compute_fold_axis(n1, c1, n2, c2))
    v1_to_hinge = sp.Point(compute_direction_toward_hinge_along_limb_1(n1 , f_axe))
    # Define symbolic variables
    k1 = sp.symbols('k1')
    C1 = sp.Matrix(c1)  # Numeric values for C1
    C2 = sp.Matrix(c2)  # Numeric values for C2
    V1 = sp.Matrix(v1_to_hinge)  # Numeric values for V1
    N2 = sp.Matrix(n2)  # Numeric values for n2
    # Define the equations
    equation1 = C1 + k1 * V1
    equation2 = (equation1 - C2).dot(N2)
    equation3 = (C1 + k1 * V1 - C2).dot(N2)
    # Solve for k1
    solution = sp.solve(equation3, k1)[0]

    return np.array(list(map(float,(np.array(Point(equation1.subs(k1, solution)))))))
    
  
def compute_intersection_point(plane1_center, plane1_normal, plane2_center, plane2_normal):
    # Convert input to NumPy arrays for easier vector operations
    plane1_center = np.array(plane1_center)
    plane1_normal = np.array(plane1_normal)
    plane2_center = np.array(plane2_center)
    plane2_normal = np.array(plane2_normal)

    # Find the direction vector of the line of intersection
    fold_axis = compute_fold_axis(plane1_normal, plane1_center, plane2_normal, plane2_center)
    v2_to_hinge = compute_direction_toward_hinge_along_limb_2(plane2_normal, fold_axis)
    c1_c2 = plane2_center - plane1_center
    k = -np.dot(c1_c2, plane1_normal)/np.dot(plane1_normal, v2_to_hinge)
    intersection_point = plane2_center  + k * np.array(v2_to_hinge)
    return intersection_point

def compute_square_side(square_nodes):
    return  np.linalg.norm(square_nodes[0] - square_nodes[1])

def compute_endpoint(start_point, direction_vector, distance):
    direction_vector = np.array(direction_vector)
    direction_vector = direction_vector/np.linalg.norm(direction_vector)
    endpoint = [float(start_point[i] + distance * direction_vector[i]) for i in range(len(start_point))]
    return endpoint

def create_finite_axial_surface(intersection_point,  axial_surf_normal, fold_axis, 
                                half_diag_size = None, 
                                 limb_1 = None, limb_2 = None):
    """method to create a finite axial surface based on a plane normal, fold axis, and if possible
      nodes of limbs for giving it a proportional size, or it will generate a random value between 10 and 100. 
      This method could be used to create any square surface based on:
        a plane normal and a direction vector of one of the four sides instead of the fold axis, and by a giving a specific half_diag_size """
    
    if half_diag_size != None :
        half_diag_size = half_diag_size
    else:    
        if (limb_1, limb_2) == (None, None):
            half_diag_size = np.random.uniform(10, 100)
        elif limb_1 != None and limb_2 != None:
            half_diag_size = 5* max(compute_square_side(limb_1), compute_square_side(limb_2))
        else:
            half_diag_size = 5*(compute_square_side(limb_1) if 
                        limb_1 is not None else compute_square_side(limb_2))    

    # Calculate the direction vectors for the sides of the square
    side1 = np.cross(axial_surf_normal, fold_axis)
    side2 = fold_axis

    # Calculate the coordinates of the four corners
    corner1 = intersection_point - half_diag_size * side1
    corner3 = intersection_point + half_diag_size * side1
    corner2 = intersection_point - half_diag_size * side2
    corner4 = intersection_point + half_diag_size * side2
    vertices = np.array([corner1, corner2 , corner3, corner4])
    translated_vertices = vertices - intersection_point
    rotation = R.from_rotvec(np.pi / 4 * axial_surf_normal)
    #Rotate the square's vertices while keeping its center fixed
     
    rotated_vertices = rotation.apply(translated_vertices)
    vertices = rotated_vertices + intersection_point
    return  vertices

def compute_s_vector(n1, c1 , n2 , c2):

    pp = np.cross(n1, n2)
    pp_norm = np.linalg.norm(pp)
    if pp_norm == 0:
        raise MalissiaBaseError("Limbs are parallel.")
    pp = pp/pp_norm
    t = n1 + n2
    t /= np.linalg.norm(t)
    c = c2 - c1
    s = c - np.dot(c,t)*t - np.dot(c,pp)*pp
    s /= np.linalg.norm(s)
    return s

def compute_fold_axis(n1, c1, n2,  c2):
    s_vector = compute_s_vector(n1, c1, n2,  c2)
    t = n1+n2
    t /= np.linalg.norm(t)
    fold_axis = np.cross(t, s_vector)
    fold_axis /= np.linalg.norm(fold_axis)
    return fold_axis

def compute_direction_toward_fold_axis(fold_axis, limb_normal):
    # Calculate the vector connecting the point to its projection
    direction_toward_fold_axis = np.cross(  fold_axis,  limb_normal)
    direction_toward_fold_axis /= np.linalg.norm(direction_toward_fold_axis)
    return direction_toward_fold_axis



In [ ]:
np.cross([1,0,0], [-1,0,0])

In [ ]:
import itertools
import numpy as np


def rotate_vector_around_axis(n1, p, theta_rad ):
    n1 = n1/ np.linalg.norm(n1)
    # Calculate the rotation matrix
    # Normalize the direction vector p over which the rotation will be performed
    p = p / np.linalg.norm(p)
    
    # Calculate the cross product matrix of p
    P = np.array([
        [0, -p[2], p[1]],
        [p[2], 0, -p[0]],
        [-p[1], p[0], 0]
    ])
    
    # Calculate the rotation matrix using the Rodriguez formula
    R = np.eye(3) + np.sin(theta_rad) * P + (1 - np.cos(theta_rad)) * np.dot(P, P)

    # Step 2: Apply the rotation to n1
    rotated_n1 = np.round(np.array(np.dot(R, n1)),8) 

    return rotated_n1 / np.linalg.norm(np.dot(R, n1))

def choose_square_side(limb_nodes):

    z = limb_nodes[:,2]
    dz1 = np.abs(z[1] - z[0])
    dz2 = np.abs(z[2] - z[1])

    if dz1 < dz2:
        selected_indices = [0,2]
    elif dz1 > dz2:
        selected_indices = [1,3]
    else:
        selected_indices = np.arange(4)

    return random.choice(selected_indices)

def get_square_side(limb_nodes, i):
    edges = np.array([[0,1],[1,2],[2,3],[3,0]])
    nodes = limb_nodes[edges[i]]
    vec = nodes[1] - nodes[0]
    return [nodes[0], nodes[1], vec]

def compute_shape_limb1(start_point, fold_axis, side_size,  limb_normal):

    vertex1 = start_point
    vertex2 = compute_endpoint(vertex1, fold_axis, side_size)
    vertex3 = compute_endpoint(vertex2, 
                               (-compute_direction_toward_hinge_along_limb_1(limb_normal,
                                fold_axis)), side_size)
    vertex4 = compute_endpoint(vertex3, -fold_axis, side_size)
    return [vertex1, vertex2, vertex3, vertex4]   

def compute_shape_limb2( vertex1, vertex2 , fold_axis, side_size,  limb_normal):

    vertex1 = vertex1
    vertex2 = vertex2
    vertex3 = compute_endpoint(vertex2, 
                               (-compute_direction_toward_hinge_along_limb_2(limb_normal,
                                fold_axis)), side_size)
    vertex4 = compute_endpoint(vertex3,fold_axis, side_size)
    return [vertex1, vertex2, vertex3, vertex4]   

def compute_distance_to_vector(point, vector_origin, vector_orientation):
    # Convert input to NumPy arrays for easier vector operations
    point = np.array(point)
    vector_origin = np.array(vector_origin)
    vector_orientation = np.array(vector_orientation)

    # Calculate the vector from the center to the point
    vector_to_point = point - vector_origin

    # Calculate the projection of vector_to_point onto vector_orientation
    projection = np.dot(vector_to_point, vector_orientation) / np.dot(vector_orientation, vector_orientation) * vector_orientation

    # Calculate the distance between the point and the projection
    distance = np.linalg.norm(vector_to_point - projection)

    return distance

def compute_point_projection_onto_line(point_to_project, line_origin, line_vector):
    # Normalize the line vector
    line_normalized = np.array(line_vector / np.linalg.norm(line_vector))
    # Calculate the vector from the origin to the point
    w = np.array(np.array(point_to_project) - line_origin)
    # Project w onto the line
    projection_length = np.dot(w, line_normalized)
    # Find the projected point on the line
    projected_point = line_origin + projection_length * line_normalized
    return np.array(projected_point)

def compute_projection_info (projected_points,  fold_axis):

    # Initialize variables to store extreme points and distances
    max_distance = -1
    extreme_points_max = None
    # Calculate pairwise distances and the extreme points
    for pair in itertools.combinations(projected_points, 2):
        point1, point2 = pair
        distance = np.linalg.norm(np.array(point1) - np.array(point2))
        if distance > max_distance:
            max_distance = distance
            extreme_points_max = [point1, point2]
    point0, point1 = np.array(extreme_points_max[0]), np.array(extreme_points_max[1])
    if all(point0 == point1):
        raise MalissiaBaseError ('the projected points coincide')
    side_size = max_distance
    vec = np.array([point1[0] - point0[0], 
           point1[1] - point0[1],
             point1[2] - point0[2]])
    parallelism = np.rad2deg(np.arccos(np.dot(vec, fold_axis) / (np.linalg.norm(vec) 
                                                        * np.linalg.norm(fold_axis))))

    if  parallelism != 0 and parallelism != 180:
        raise MalissiaBaseError('projected points on the fold axis are not collinear')
    if np.dot(vec, fold_axis) > 0:
        start_point = point0
        end_point = point1
    else: 
        start_point = point1
        end_point = point0

    return {'start_point': start_point, 'end_point': end_point,
             'side_size': side_size}



def compute_fold_geometry_from_two_limbs(limb1, limb2):
    n1 = calculate_square_normal(*limb1)
    n2 = calculate_square_normal(*limb2)
    c1 = compute_square_center(*limb1)
    c2 = compute_square_center(*limb2)
    fold_axis = compute_fold_axis(n1, c1, n2, c2)
    intersection_point = compute_intersection_point(c1, n1,c2, n2)
    all_surfaces_nodes = [v for v in limb1]
    for v in limb2:
        all_surfaces_nodes.append(v) 
    projected_nodes = []
    for node_i in all_surfaces_nodes:
        projected_nodes.append(compute_point_projection_onto_line(node_i, 
                intersection_point, fold_axis))
    information = compute_projection_info (projected_nodes,  fold_axis)
    all_distances = [compute_distance_to_vector(n_i, intersection_point, fold_axis) 
                    for n_i in all_surfaces_nodes]
    max_distance = np.max(all_distances)
    side_size =  information['side_size'] if information['side_size']>= max_distance else  max_distance
    
    new_limb1 = compute_shape_limb1(information['start_point'],
                                    fold_axis, 
                                    side_size,
                                    n1)
    vertex1_for_2limb, vertex2_for_2limb = new_limb1[1],new_limb1[0]
    new_limb2 = compute_shape_limb2(vertex1= vertex1_for_2limb,
                                    vertex2= vertex2_for_2limb,
                                    fold_axis= fold_axis, 
                                    side_size = side_size,
                                    limb_normal= n2)    
    axial_surf_normal = compute_axial_surface_normal(n1 ,n2, fold_axis)
    midpoint = np.array([(i1 + i2) / 2 for i1, i2 in zip(new_limb1[1],new_limb1[0])]) 
    finite_axial_surf = create_finite_axial_surface(midpoint,  axial_surf_normal, 
                                                    fold_axis, 
                                                    half_diag_size = information['side_size']*5)
    intersection_point = midpoint
    return {'fold_axis': fold_axis, 
            'axial_surf_normal': axial_surf_normal, 
            'finite_axial_surf': finite_axial_surf,
            'limb1': new_limb1,
            'limb2': new_limb2, 
            'intersection_point': intersection_point}



def compute_complement_angle(angle_radians):

    # Calculate the complement in radians
    complement_radians = math.pi - angle_radians
    
    return  complement_radians


def compute_fold_geometry_from_one_limb(candidate_limb_nodes,  
                                                 force_limb_assignment = None , 
                                                 force_fold_type = None , 
                                                 chosen_side_index = None,
                                                 fold_opening_angle_degree  = 90. ):
    
    """method for building a fold based on a candidatefold limb. 
    for all arguments  if none is chosen, the algorithm will make a random generation of the paramters: 
    - force_limb_assignment must be int 1 or 2  
    - fold type must be string anticline or aynscline 
    - chosen_side_index must be an int from 0 to 3 
    - fold_opening_angle_degree must be flaot from 0.0001 to 179.9999
    """
    
    candidate_limb_nodes = np.array(candidate_limb_nodes)
    initial_limb = force_limb_assignment if force_limb_assignment is not None else random.choice([1,2])
    if initial_limb not in (1,2): 
        raise MalissiaBaseError('The choice of which limb could not be be made, try giving force_limb_assignment a value of 1 or 2')

    fold_type = force_fold_type if force_fold_type is not None else random.choice(['anticline','syncline'])
    if fold_type not in ['anticline','syncline']: 
        raise MalissiaBaseError('The choice of fold type could not be made, try giving force_limb_assignment a value of anticline syncline')


    chosen_side_index = chosen_side_index if chosen_side_index is not None else choose_square_side(candidate_limb_nodes)
    chosen_side = get_square_side(candidate_limb_nodes, chosen_side_index)
    if len (chosen_side) == 0 :
        raise MalissiaBaseError('the algorithm could not chose a square side')
    
    midpoint_side = np.mean((chosen_side[0], chosen_side[1]), axis=0)
    vector_to_axis = midpoint_side - np.array(compute_square_center(*candidate_limb_nodes))


    initial_normal = calculate_square_normal(*candidate_limb_nodes)
    
    proposed_fold_axis = np.cross(initial_normal, vector_to_axis) / np.linalg.norm(np.cross(initial_normal, vector_to_axis))



    if initial_limb == 1:
        fold_opening_angle_degree = fold_opening_angle_degree if fold_type == 'anticline' else -fold_opening_angle_degree

    if initial_limb == 2:
        proposed_fold_axis *= -1
        fold_opening_angle_degree = fold_opening_angle_degree if fold_type == 'syncline' else -fold_opening_angle_degree

    #fold_opening_angle_degree = 2*dip_compliment_from_self.object if fold_opening_angle_degree is None else fold_opening_angle_degree
    angle_of_normal_rotation_rad =  compute_complement_angle(np.deg2rad(fold_opening_angle_degree))
    

    computed_rotated_normal = rotate_vector_around_axis(initial_normal, 
                                                        proposed_fold_axis, 
                                                        theta_rad= angle_of_normal_rotation_rad)
    side_size  = np.linalg.norm(chosen_side[1] - chosen_side[0])

    if initial_limb == 1:
        limb1_nodes = candidate_limb_nodes
        limb2_v1 = chosen_side[1]
        limb2_v2 = chosen_side[0]
        direction_toward_hinge_2 = compute_direction_toward_hinge_along_limb_2(computed_rotated_normal, proposed_fold_axis)
        limb2_v3 = compute_endpoint(limb2_v2, -direction_toward_hinge_2, side_size)
        limb2_v4 = compute_endpoint(limb2_v3, proposed_fold_axis, side_size)
        limb2_nodes = np.array([limb2_v1, limb2_v2, limb2_v3, limb2_v4])

    if initial_limb == 2:
        limb2_nodes = candidate_limb_nodes
        limb1_v1 = chosen_side[1]
        limb1_v2 =chosen_side[0]
        direction_toward_hinge_1 = compute_direction_toward_hinge_along_limb_1(computed_rotated_normal, proposed_fold_axis)
        limb1_v3 = compute_endpoint(limb1_v2, -direction_toward_hinge_1, side_size)
        limb1_v4 = compute_endpoint(limb1_v3, -proposed_fold_axis, side_size)
        limb1_nodes = np.array([limb1_v1, limb1_v2, limb1_v3, limb1_v4]) 
    # now generate fold paramters    
    n1 = calculate_square_normal(*limb1_nodes)
    n2 = calculate_square_normal(*limb2_nodes)
    c1 = compute_square_center(*limb1_nodes)
    c2 = compute_square_center(*limb2_nodes)
    intersection_point = compute_intersection_point(c1, n1,c2, n2)  
    axial_surf_normal = compute_axial_surface_normal(n1 ,n2, proposed_fold_axis)
    finite_axial_surf = create_finite_axial_surface(intersection_point,  axial_surf_normal, 
                                                    proposed_fold_axis, 
                                                    half_diag_size = side_size*1.5)
    return {'fold_axis': proposed_fold_axis, 
            'axial_surf_normal': axial_surf_normal, 
            'finite_axial_surf': finite_axial_surf,
            'limb1': limb1_nodes,
            'limb2': limb2_nodes, 
            'intersection_point': intersection_point}



In [ ]:
vertices_1= [
    np.array([-6, 0, 0]),
    np.array([-10, 0, 4]),
    np.array([-10.0, 6.0, 4.0]),
    np.array([-6.0, 6.0, 0.0])
]
vertices_2 = [
    np.array([10, -3.0, -4]),
    np.array([10, 0, -4]),
    np.array([15.0, 0.0, 0.0]),
    np.array([15, -3.0, 0.0])
]


x = compute_fold_geometry_from_two_limbs(vertices_1, vertices_2)



In [ ]:
vertices_1= [
    np.array([0., 0, 0]),
    np.array([0., -5, 0]),
    np.array([4., -5, 4]),
    np.array([4., 0, 4])
]
theta = 5

x = compute_fold_geometry_from_one_limb(candidate_limb_nodes = vertices_1, 
                                        force_limb_assignment = 1, 
                                        force_fold_type= 'syncline',
                                        fold_opening_angle_degree= theta,
                                          chosen_side_index= 2)


In [ ]:

import pyvista  as pv


#plane = pv.Plane()
#line = pv.Line(np.array([1,5,1]), np.array([0,1,0]))
#line1 = pv.Line(np.array([0,0,0]), np.array([0,0,5]))
#poly = pv.Polygon()
#points = np.array([np.array([1,1,1]), np.array([5,1,1])])
#p.add_points( points, render_points_as_spheres=True, point_size=5.0 )


vertices_x = np.array(x['limb1'])
arrow1 = pv.Arrow(compute_square_center(*x['limb1']), calculate_square_normal(*x['limb1']))
faces1 = np.hstack([ [ 4, 0, 1, 2, 3]])
surf1 = pv.PolyData(vertices_x, faces1)


vertices_y = np.array(x['limb2'])
arrow2 = pv.Arrow(compute_square_center(*x['limb2']), calculate_square_normal(*x['limb2']))
faces2 = np.hstack([[ 4, 0, 1, 2, 3]])
surf2 = pv.PolyData(vertices_y, faces2)


vertices_ax = np.array(x['finite_axial_surf'])
arrow3 = pv.Arrow(compute_square_center(*x['finite_axial_surf']), x['axial_surf_normal'])
faces3 = np.hstack([ [ 4, 0, 1, 2, 3]])
surf3 = pv.PolyData(vertices_ax, faces3)



p = pv.Plotter()

#arrow_d1= pv.Arrow(compute_square_center(*x['limb1']), x['d1'])
#arrow_d2 = pv.Arrow(compute_square_center(*x['limb2']), x['d2'])
#computed_rotated_normal = pv.Arrow([-3,-3,-3], x['computed_rotated_normal'])

p.add_mesh(arrow1, color='red', show_edges=True)
p.add_mesh(arrow2, color='red', show_edges=True)
#p.add_mesh(arrow3, color='red', show_edges=True)
####
#p.add_mesh(arrow_d1, color='red', show_edges=True)
#p.add_mesh(arrow_d2, color='red', show_edges=True)
#p.add_mesh(computed_rotated_normal, color='black', show_edges=True)
####

p.add_mesh(surf1, color='grey', show_edges=True)
p.add_mesh(surf2, color='yellow', show_edges=True)
#p.add_mesh(surf3, color='black', show_edges=True)

# Render all of them
p.show()

In [ ]:

## useful methods 


def check_sides_parallel_to_fold_axis(limb1_nodes, limb2_nodes):
    """Method to check if at least one side of the two limbs sqare surfaces is
      parallel to the fold axis or not"""

    n1 = calculate_square_normal(*limb1_nodes)
    n2 = calculate_square_normal(*limb2_nodes)
    fold_axis = compute_fold_axis(n1, n2)
    axial_surf_normal = compute_axial_surface_normal(n1, n2, fold_axis)
    p_intersection = compute_intersection_point(limb1_nodes, limb2_nodes)
    axial_plane = sp.Plane(sp.Point(p_intersection), sp.Point(axial_surf_normal))
    
    distance1 = np.sort([float(axial_plane.distance(sp.Point(node_i))) for node_i in limb1_nodes])
    distance2 = np.sort([float(axial_plane.distance(sp.Point(node_i))) for node_i in limb2_nodes])
    limbs_not_parallel_to_fold_axis = []
    if distance1[0] == distance1[1]:
        limbs_not_parallel_to_fold_axis.append(limb1_nodes)
    if distance2[0] == distance2[1]:
        limbs_not_parallel_to_fold_axis.append(limb2_nodes)
    #limbs_not_parallel_to_fold_axis = [limb for limb in []]
    return limbs_not_parallel_to_fold_axis


def compute_point_project_onto_plane(point_to_project, vector1, vector2, point_on_plane):
    # Calculate the unit normal vector
    unit_normal = np.cross(vector1,
                            vector2) / np.linalg.norm(np.cross(vector1, vector2))
    # Calculate the vector from the given point to the point on the plane
    vector_to_plane = point_to_project - point_on_plane
    # Calculate the projection of vector_to_plane onto the unit_normal vector
    projection = np.dot(vector_to_plane, unit_normal)
    # Calculate the projected point on the plane
    return point_to_project - projection * unit_normal
    

def compute_most_distant_point_limbs(limb1_nodes, limb2_nodes):
    most_distant_point_list1 = None
    most_distant_point_list2 = None
    max_distance = -1  # Initialized to a negative value, so any distance will be greater
    most_distant_point_list1 = None
    most_distant_point_list2 = None
    max_distance = -1  # Initialized to a negative value, so any distance will be greater

    # Iterate over each combination of points from list1 and list2
    for point1 in limb1_nodes:
        for point2 in limb2_nodes:
            distance = np.linalg.norm(point1 - point2)  # Calculate the distance between the points
            if distance > max_distance:
                max_distance = distance
                most_distant_point_list1 = point1
                most_distant_point_list2 = point2
    return list([most_distant_point_list1, most_distant_point_list2])
    

def compute_s_vector_limb1_toward_limb2(limb1_normal, point_on_limb1,
                                         limb2_normal, point_on_limb2):
    t = (limb1_normal +limb2_normal) / np.linalg.norm(limb1_normal +limb2_normal)
    
    # use for now 'compute_fold_axis()' but need making it compute p prime
    pp = compute_fold_axis_non_oriented(limb1_normal, limb2_normal)
    # use the point of the second limb as a center of the plane that is parallel to the axial plane
    # project the point on the limb1 on this plane
    projected_point = compute_point_project_onto_plane(point_to_project = point_on_limb1 ,
                                                        vector1 = t , 
                                                        vector2 = pp, 
                                                        point_on_plane = point_on_limb2)
    # return the vector between the projection and the inital point on limb1 as the vector pointing to the limb2
    return np.array((projected_point-point_on_limb1 ) / np.linalg.norm(projected_point-point_on_limb1))
     

def compute_plane(point, normal):
    return Plane(Point(point), Point(normal))


def compute_working_direction(limb_nodes,  preferential_diretion = 'x'):
    if preferential_diretion == 'x':
        return np.array([1,0,0])
    else:
        diag_distance = np.linalg.norm((np.array(limb_nodes[0])-np.array(limb_nodes[2])))
        diag_distance = diag_distance*1.3
        random_direction = np.random.rand(3) - 0.5  # Create a random vector in the range [-0.5, 0.5] for each component
        random_direction /= np.linalg.norm(random_direction)  
        center = compute_square_center(*limb_nodes)
        new_point = center + diag_distance * random_direction
        plane_normal = calculate_square_normal(*limb_nodes)
        while np.abs(np.dot(new_point - center, plane_normal)) < 1e-6:
        # Adjust the new point if it's still on the plane (within a small tolerance)
            new_point += 1e-5 * random_direction
        working_direction = np.array(new_point-center)
        return working_direction
    

def generate_point_away_from_plane(original_point, plane_normal, distance, direction_vector):
    # Ensure the direction vector is normalized
    direction_vector = direction_vector / np.linalg.norm(direction_vector)

    # Calculate the new point
    new_point = original_point + distance * direction_vector

    # Ensure the new point is not on the plane
    if np.abs(np.dot(new_point - original_point, plane_normal)) < 1e-6:
        # Adjust the new point if it's still on the plane (within a small tolerance)
        new_point += 1e-6 * direction_vector

    return new_point
 

## Representation Spaces

We distinguish two kind of operations here:
* representation
* visualisation

A representation is a formal description of how something appears in a given representation space, but it doesn't have to be visualised.<br>
A visualisation takes care of the rendering of a representation with a given support (image, screen).

Representation should also be made a bit more abstract.<br>
1. There is a variety of object that can be rendered in a representation space (typically, different kinds of a dataset components)
2. Several kinds of representation spaces could be envisionned (e.g., spatial 1D,2D,3D, or temporal, or just an abstract text)

In [ ]:

import numpy as np
import pyvista as pv
from scipy import linalg

class RepresentationSpace(object):
    """A general framework for Representating geological objects"""
    
    def __init__(self, knowledge_framework = None):
        """Initialise base class attributes"""
        self.knowledge_framework = knowledge_framework if knowledge_framework is not None else GeologicalKnowledgeManager().get_knowledge_framework()
        self._datasets= []
        self._interpreted_objects = []
        
    def attach_dataset(self, dataset, use_extension= True, padding= None):
        """Attach a dataset to the representation space
        
        Parameters:
        - dataset: a Dataset object to be attached
        - use_extension: if True, uses the extension of the dataset, else keeps the current ones
        - padding: if use_extension is True, the given paddign will be used to keep a space around the dataset"""
        if dataset == None: return
        if dataset in self._datasets: return
        self._datasets += [dataset]
        
        if use_extension:
            self.set_extension_from_data(padding= padding)
        
        dataset.setup_representation_space(physical_space= self)
    
    def get_datasets(self):
        """dataset getter"""
        return self._datasets
        
    def attach_interpreted_object(self, interpreted_object, update_extension= True, padding= None):
        """Attach an interpreted object to the representation space
        
        Parameters:
        - interpreted_object: an interpreted object  to be attached
        - update_extension: if True, uses the extension of the interpreted object, else keeps the current ones
        - padding: if use_extension is True, the given padding will be used to keep a space around the interpreted object"""
        if interpreted_object == None: return
        if interpreted_object in self._interpreted_objects: return
        self._interpreted_objects += [interpreted_object]
        
        if update_extension:
            # Todo self.update_extension_from_interpreted_object(interpreted_object, padding= padding)
            print("Warning: extensionupdate from interpreted object is not implemented yet")
    
    def get_interpreted_objects(self):
        """interpreted objects getter"""
        return self._interpreted_objects
        
class TemporalRepresentationSpace(RepresentationSpace):
    """A `RepresentationSpace` representing temporal apsects of represented objects."""
    
class PhysicalRepresentationSpace(RepresentationSpace):
    """A type of `RepresentationSpace` representing physical aspects of the represented objects."""
    
    __default_coordinate_labels = ["X","Y","Z"]
    
    def __init__(self, dimension: int=None, coordinate_labels: str|list= None, dataset= None, **kargs):
        """Initialisation of the representation space.
        
        Parameters:
        - dimension (int): specify the number of dimensions of the representation space, typically 1D, 2D, or 3D (i.e., 1, 2, or 3),
        NB: larger dimension spaces are not supported. At least either the `dimension` parameter or `coordinate_label` parameter should be given.
        - coordinate_labels(str|list(str)): gives the label(s) of the coordinates. If given, the number of dimensions is deduced from the size of the list
        and `dimensions`is ignored, otherwise, the labels are taken from the `__default_coordinate_labels` based on the number of `dimension`s. 
        At least either the `dimension` parameter or `coordinate_label` parameter should be given.
        - dataset: a Dataset object containing the data to be attached to this representation space.
        Note that the RepresentationSpace can be created first and then updated automatically when creating the dataset attached to this space.
        - **kargs:
            - use_extension: if True, uses the extension of the dataset, else keeps the current ones
            - padding: if use_extension is True, the given paddign will be used to keep a space around the dataset
        """       
        super().__init__() 
        
        assert not (coordinate_labels is None and dimension is None), "At least one of the parameters should be specified"
        if coordinate_labels is None:
            assert isinstance(dimension, int),"dimension parameter must be an integer"
            assert dimension in [1,2,3], "The specified number of dimensions ({:d}) is not supported, should be 1, 2 or 3.".format(dimension)
            self.dimension= dimension
            self.coordinate_labels= PhysicalRepresentationSpace.__default_coordinate_labels[:self.dimension]
        elif isinstance(coordinate_labels,str):
            self.dimension= 1
            self.coordinate_labels=  [coordinate_labels]
        elif isinstance(coordinate_labels, list):
            self.dimension= len(coordinate_labels)
            self.coordinate_labels= coordinate_labels
        else:
            raise("Unsupported initialisation of representation space: dimension({}) and coordinate_label ({}).\n At least one of the parameters shoudl be specified.".format(dimension, coordinate_labels))

        self.__default_padding= 0.1
        self.set_extension()
        self.attach_dataset(dataset,**kargs)
    
    def get_extension(self):
        """returns the extensions of the space"""
        return self.extension
    
    def get_size(self):
        """returns the size of the space in each dimension"""
        ext = np.array(self.extension)
        return ext[:,1] - ext[:,0]
    
    def get_center(self):
        """returns the center of the space"""
        return np.mean(self.extension, axis= 1)
    
    def generate_random_location(self, n= 1):
        """Generates n random location uniformly distributed within the space extensions"""
        return np.random.default_rng().uniform(low= self.extensions[:0], high= self.extensions[:1], size= n)
    
    def generate_random_dip(self, n= 1):
        """Generates n random dip values that are compatible with this space"""
        return np.random.default_rng().uniform(0, 90, n)

    def generate_random_dip_dir(self, n= 1):
        """Generates n random dip_dir values that are compatible with this space
        
        Note: The values are hardcoded to be in XZ cross section to simplify the example"""
        return np.random.default_rng().choice([90,270], n)#np.random.default_rng().uniform(0, 360, n)
    
    def generate_random_polarity(self, n= 1):
        """Generates n random polarity values (-1 or 1)"""
        return np.random.default_rng().choice([-1,1], n)
    
    def generate_random_size(self, n=1, min_perc= 0.05, max_perc= 0.99):
        """Generates n random size values"""
        sizes = self.get_size()
        max_sizes = max(sizes)
        return np.random.default_rng().uniform(min_perc*max_sizes, max_perc*max_sizes, n)
        
    def set_extension(self, extension:list= None):
        """Setter for the extension (min,max) of the representation space
        
        Parameters:
        - extension: a list containing a pair of min and max value for each dimension of the space.
        If None, the default will be set, i.e., [[0,1]] * dimension"""
        if extension is None: 
            self.extension = [[0,1]] * self.dimension
            return
        extension= np.array(extension)
        assert extension.shape[0] == self.dimension, "The specified extension ({}) do not match the space dimensions ({})".format(extension, self.dimension)
        assert extension.shape[1] == 2, "The specified extension should provide both lower andupper bounds for each dimension, given: {}".format(extension)
        self.extension= extension
        
    def set_extension_from_data(self, padding= None):
        """Sets the extension of the space from the attached dataset
        
        If no dataset is attached yet, then default extension are used instead (min:0,max:1).
        
        Parameters:
        - padding: a space that is left around the dataset, either a value compatible with the coordinates, or a list of values of same dimensions.
        If None, by default the padding is 5% of the dataset range.
        """
        non_empty_dataset = [data_i for data_i in self._datasets if data_i.extension is not None]
        if len(non_empty_dataset) == 0:
            self.set_extension()
            return
        
        if padding is None:
            padding= self.__default_padding
        else:
            try: # check if padding as a dimension
                len(padding)
            # if not, then use it a a scaling 
            except TypeError: #just checking it is a number
                assert type(padding) == int or type(padding) == float, "padding should be given as a number (int or float), here: "+type(padding)
                # keep the padding as is in this case
            else:# else check its dimensions are ok
                assert len(padding) == self.dimension, "the dimensions of the specified padding (len({})->) should match the space dimension ({})".format(padding, len(padding), self.dimension)
                padding= np.array(padding)
                
        extension = non_empty_dataset[0].extension
        for data_i in non_empty_dataset[1:]:
            for dim_i in self.dimension:
                extension[dim_i,0] = min(extension[dim_i,0], data_i.extension[dim_i])
                extension[dim_i,1] = max(extension[dim_i,1], data_i.extension[dim_i])
        """:todo: use projected coordinates instead of source coordinates, might fail if 3D data projected on a map"""
                
        # in any cases, except when padding and data is None
        self.center= np.mean(extension, axis= 1)
        diff= extension.T - self.center
        diff= diff.T
        
        # padding is multiplied by the width along each axis and added to the extension
        # check if each dimension is shrinked (ie. min == max -> diff == 0), takes the average extension of the other dimensions,
        # unless all are shrinked, in which case [[-1,1]] * self.dimension is set
        if np.all(diff == 0):
            diff = np.array([[-1,1]] * self.dimension)
        else:
            zero_diff = np.where(~diff.any(axis=1))
            non_zero_diff = np.where(diff.any(axis=1))
            mean_diff = np.mean(diff[non_zero_diff], axis=0)
            diff[zero_diff] = mean_diff
        
        extension= np.repeat([self.center],2,axis=0).T + (1+2*padding)*diff 
        
        self.set_extension(extension)
        
    def filter_qualities(self,**qualities):
        """removes the named arguments corresponding to this space coordinates from qualities.
        
        Return:
        - a copy with the passed parameters except for the ones correpsonding to the space coordinates"""
        qualities =  {key:val for key, val in qualities.items() if key not in self.coordinate_labels}
        return qualities
    
    def prepare_coordinate_qualities(self, **qualities):
        """transforms the coordinates passed in qualities into an appropriate format
        
        Return:
        - a dict with quality keys and values for setting coordinates"""
        coordinates = {key:val for key, val in qualities.items() if key in self.coordinate_labels}
        coord_qualities = {}
        for i,key in enumerate(self.coordinate_labels):
            if key in coordinates:
                val = coordinates[key]
                coord_qualities["coord{}".format(i+1)] = val if isinstance(val,list) else [val]
                coord_qualities["coord{}_label".format(i+1)] = [key]
        return coord_qualities
    
    def label_map(self, object):
        """creates of mapping of coordinate labels"""
        coord_label_params = ["coord{}_label".format(i) for i in range(1,4) if hasattr(object, "coord{}_label".format(i))]
        non_empty_coord_label_param = [param for param in coord_label_params if len(getattr(object,param)) > 0]
        label_map = {getattr(object,param)[0]:param for param in non_empty_coord_label_param if getattr(object,param)[0] in self.coordinate_labels}
        return label_map
        
    def set_object_coordinates(self, object, **kargs):
        """Sets the coordinates corresponding to this space into the given object
        
        Parameters:
        - object: the object whose coordinates needs to be set
        - kargs: keyword argugments corresponding to the name and values of the coordinates.
        They must match this space coordinate names, extra names will be ignored"""
        
        # find the object representation and check it is a point
        #otherwise another setter must be used
        if (object.has_Representation is None) or (len(object.has_Representation) < 1):
            raise MalissiaBaseError("can't set the object coordinates because it doesn't have a geometrical representation")
        rep = object.has_Representation[0]
        if not self.knowledge_framework.isinstance(rep, self.knowledge_framework().Point):
            raise MalissiaBaseError("can't set the object coordinates because its geometrical representation is not a Point")            
        
        # check if the object has coord{i}_label set, else set it
        if mogi.has_qualities(rep, ["coord{}_label".format(i) for i in range(1,len(self.coordinate_labels)+1)]):
            label_map = self.label_map(rep)
            coordinate_values = {label_map[key].split("_")[0]:(kargs[key] if isinstance([kargs[key]],list) else [kargs[key]])
                                  for key in self.coordinate_labels if key in kargs}
        else:
            coordinate_values = self.prepare_coordinate_qualities(**kargs)
        for param_name, val in coordinate_values.items():
            setattr(rep, param_name, val)
            
    def get_object_coordinates(self, object):
        """Gets the coordinates corresponding to this space from the given object"""
        
        # find the object representation and check it is a point
        #otherwise another setter must be used
        if (object.has_Representation is None) or (len(object.has_Representation) < 1):
            raise MalissiaBaseError("can't get the object coordinates because it doesn't have a geometrical representation")
        rep = object.has_Representation[0]
        if not self.knowledge_framework.isinstance(rep, self.knowledge_framework().Point):
            raise MalissiaBaseError("can't get the object coordinates because its geometrical representation is not a Point") 
        
        label_map = self.label_map(rep)
        coord = np.full_like(self.coordinate_labels, np.nan, dtype= float)
        for i, key in enumerate(self.coordinate_labels):
            if key in label_map:
                coord_param = label_map[key].split("_")[0]
                coord[i] = getattr(rep,coord_param)[0]
        return coord
    
    def coordinates_to_dict(self, coords):
        """ transforms a coordinate array into an appropriate dict with labels"""
        return {label:value for label,value in zip(self.coordinate_labels, coords)}
    
    def get_normal_vector(self, feature):
        dip = feature.dip
        dip_dir = feature.dip_dir
        return self.compute_normal_from_dip_dir(dip, dip_dir)
        
    def compute_normal_from_dip_dir(self, dip, dip_dir, polarity= 1):
        dip_rad = np.deg2rad(dip)
        dip_dir_rad = np.deg2rad(dip_dir)
        z = np.cos(dip_rad)
        h = np.sin(dip_rad)
        y = h * np.cos(dip_dir_rad)
        x = h * np.sin(dip_dir_rad)
        return polarity * np.array([x,y,z])
    
    def compute_dip_dir_from_normal(self, normal):
        polarity = 1 if normal[2] == 0 else np.sign(normal[2]) 
        x,y,z = polarity * np.array(normal) / np.linalg.norm(normal)
        if z == 1:
            return 0,0
        dip = np.rad2deg(np.arccos(z))
        dip_dir = np.rad2deg(np.arctan2(x,y)) % 360
        return dip, dip_dir, polarity
    
    def compute_line_attitude_from_two_points(self, p0, p1, center= False):
        """Computes the attitude of a line going through two points
        
        Parameters:
        - p0: coordinates of the first point
        - p1: coordinates of the second point
        
        Returns:
        - attitude: a dictionnary holding
          "dip_dir" : the dip direction of the line. +or- 1 if dimension is  <3, value otherwise
          "dip" : the dip of the line (ie., downward). None if dimension is <2, value otherwise
        """
        v = np.array(p1) - np.array(p0)
        center = np.mean(v,axis=0) if center else np.zeros(v.shape[-1])
        if len(v) == 1:
            dip_dir = np.sign(v)[0]
            dip = None
        else:
            # make v downward
            if v[-1] > 0:
                v *= -1 
            if len(v) == 2:
                dip_dir = np.sign(v[0])
                v_abs = np.abs(v)
                dip = np.rad2deg(np.arctan2(v_abs[1], v_abs[0]))
            else:
                dip_dir = np.rad2deg(np.arctan2(v[0],v[1]))
                dip_dir = 360 - np.abs(dip_dir) if dip_dir < 0 else dip_dir
                dip = -np.rad2deg(np.arctan2(v[2], np.linalg.norm(v[:2]) ))
        size = np.linalg.norm(v)
                
        return {"dip_dir":dip_dir, "dip":dip, "center":center, "size":size}
            
    def compute_principal_directions(self, p, center= False):
        """Computes the principal directions in a set of vectors
        
        This can be used to compute the principal vectors in a set of vectors
        or medium plan in a group of points (needs to set center to True).
        
        Parameters:
        - p: points/vectors in the shape (# points, # dimensions)
        - center: tells if points should be centered first, important for computing medium plane, default is False
        
        Returns: a dictionnary with:
        - "values": the singular values of the principal axes
        - "vectors": the vectors of the principal axes
        - "center": the center of points if center is set to True, zeros otherwise
        """
        p = np.array(p)
        center = np.mean(p,axis=0) if center else np.zeros(p.shape[-1])
        p = p - center
        mat = np.dot(p.T, p)
        _,s,Vh = linalg.svd(mat)
        result = {
            "values": s,
            "vectors": Vh,
            "center": center
        }
        return result
    
    def compute_average_vector(self, vectors):
        """Compute the average vector"""
        principal_directions = self.compute_principal_directions(vectors)
        principal_vectors = principal_directions["vectors"]
        return principal_vectors[0]
    
    def compute_attitude_from_points(self, p):
        """Computes the attitude dip and dip_dir for a medium plane going through points
        
        Parameters:
        - p: the points coordinates given in shape ()
        """
        if len(p) < self.dimension:
            raise MalissiaBaseError("Underdetermined attitude computation.")
            
        if self.dimension == 2:
            return self.compute_line_attitude_from_two_points(p, center= True)
        elif self.dimension == 3:
            return self.compute_plane_from_points(p)
        else:
            raise MalissiaBaseError("unsupported dimension for attitude computation.")
        
    def compute_plane_from_points(self, p):
        """Computes the attitude dip and dip_dir for a medium plane going through points
        
        Parameters:
        - p: the points coordinates given in shape ()
        
        Returns:
        - "dip_dir": azimuth from North towards the East, None if dip is zero
        """
        principal_direction = self.compute_principal_directions(p, center= True)
        
        result = {}
        result["center"] = principal_direction["center"]
        result["major_axis"] = principal_direction["vectors"][0]
        result["minor_axis"] = principal_direction["vectors"][1]
        result["size"] = np.sqrt(principal_direction["values"][0])
        
        result["normal"] = principal_direction["vectors"][-1]
        n = -result["normal"] if result["normal"][2]<0 else result["normal"]
        h = np.linalg.norm(n[:2])
        result["dip"] = np.rad2deg(np.arctan2(h, n[2]))
        if h == 0:
            result["dip_dir"] = None
        else:
            dip_dir = np.rad2deg(np.arctan2(n[0], n[1]))
            result["dip_dir"] = 360 - np.abs(dip_dir) if dip_dir < 0 else dip_dir
        result["azimuth"] = result["dip_dir"] - 90 if result["dip_dir"] > 90 else result["dip_dir"] + 270
        return result
    
    
    def __str__(self):
        """Description of the physical space parameters and data
        """
        desc= ["Representation space of type: {:s}".format(type(self).__name__)]
        desc+= ["- Number of dimension(s): {}".format(self.dimension)]
        desc+= ["- Coordinate label(s): {}".format(self.coordinate_labels)]
        desc+= ["- Space extension:"]
        for dim_i, lim_i in zip(self.coordinate_labels,self.extension):
            desc+= [" |- Coord {}: {}".format(dim_i, lim_i)]  
        return "\n".join(desc)

## Dataset

In [ ]:
import logging

class GeologicalDataset(object):
    """A GeologicalDataset gathers information about geological data to be interpreted.
    
    This class is a hybrid ontology&python class. It is providing pythonic algorithm and high level interface,
    while the data is actually stored in an ontology.
    """
    
    def __init__(self, physical_space= None, time_space= None, representation_spaces= None, knowledge_framework= None):
        """Initialises a `GeologicalDataset`
        
        Parameters:
        - physical_space: a `PhysicalRepresentationSpace`,  which defines the spatial coordinates of this dataset
        - time_space: a `TemporalRepresentationSpace`,  which defines the time coordinates of this dataset
        - representation_spaces: list of `Representationspace`s to which the dataset must be attached
        Note: datasets can be created without representation space and attached later on by using `RepresentationSpace.attach_dataset`
        or `GeologicalDataset.setup_representation_space`.
        Alternativelly, a single `PhysicalRepresentationspace` and or `TemporalRepresentationspace` can be given here if `physical_space` and `time_space` are None.
        - default_representation_space: the main RepresentationSpace to which this dataset is attached.
        If None and representation_spaces are provided, then the first one will be taken.
        If the default one is not initially in the full list, then it is added to it.
        - ontology: the name of the ontology to be used for storing the data.
        If None, the default will be taken from `the GeologicalKnowledgeManager`.
        
        Internals: this method initialises several internal attributes:
        - extension: represents the extension of the dataset in the attached representation space
        (i.e., the default on if this dataset is represented in several representation spaces
        - representation_spaces: the dataset can be attached to and represented into several representation spaces,
        `physical_space` and `time_space` are included into this list.
        """
        
        self.knowledge_framework= knowledge_framework if knowledge_framework is not None else GeologicalKnowledgeManager().get_knowledge_framework()
        
        self.extension = None
        
        # setup representation spaces
        self.representation_spaces= set()
        if physical_space is None:
            # try to initialise the physical space with first existing data
            observations = self.get_observations()
            if len(observations) > 0:
                coord_labels = [getattr(observations[0],"coord{}_label".format(i)) for i in range(1,4)]
                coord_labels = [label[0] for label in coord_labels if len(label)>0]
                physical_space = PhysicalRepresentationSpace(coordinate_labels=coord_labels)
        self.setup_representation_space(physical_space, time_space, representation_spaces)
        
        # register the datast in the listed representation spaces
        for space_i in self.representation_spaces:
            space_i.attach_dataset(self)
        
        # initialize extension of the dataset
        self.update_extension()
            
    def __dell__(self):
        self.remove_all_observations()

    def __setup_space(self, space, space_type):
        """Check if space of given type is in list or parameter and return the appropriate value.
        
        Take the given physical/time space, or if None use the first one in the list, and if none just leave None.
        Adds the space to the `self.representation_spaces` set."""
        if space is not None: 
            self.representation_spaces.add(space)
            return space
        if len(self.representation_spaces) == 0: return None
        space_list= [space_i for space_i in self.representation_spaces if isinstance(space_i, space_type)]
        return space_list[0] if len(space_list) > 0 else None
    
    def update_extension(self, update_representation_spaces= True):
        """Sets the extension of the dataset in the physical space from existing observations
        
        If there is no observation, `self.extension` is set to None"""
        observations = self.get_observations()
        if (len(observations) == 0) or (self.physical_space is None) or (self.physical_space.dimension == 0):
            self.extension = None 
            return
            
        di = observations[0]
        coord = self.physical_space.get_object_coordinates(di)
        self.extension = np.repeat([coord],2,axis=0).T
        for di in observations[1:]:
            coord = self.physical_space.get_object_coordinates(di)
            for j, val in enumerate(coord):
                self.extension[j,0] = min(self.extension[j,0], val)
                self.extension[j,1] = max(self.extension[j,1], val)
                
        if update_representation_spaces:
            if self.physical_space is not None:
                self.physical_space.set_extension_from_data()
        
    def setup_representation_space(self, physical_space= None, time_space= None, representation_spaces= None):
        """Setup the representation space list and default
        
        Parameters:
        - physical_space: a `PhysicalRepresentationSpace`,  which defines the spatial coordinates of this dataset
        - time_space: a `TemporalRepresentationSpace`,  which defines the time coordinates of this dataset
        - representation_spaces: list of `Representationspace`s to which the dataset must be attached
        Note: datasets can be created without representation space and attached later on by using `RepresentationSpace.attach_dataset`
        or `GeologicalDataset.setup_representation_space`.
        Alternativelly, a single `PhysicalRepresentationspace` and or `TemporalRepresentationspace` can be given here if `physical_space` and `time_space` are None.
        """
        if representation_spaces is not None: self.representation_spaces = self.representation_spaces.union(representation_spaces)
        self.physical_space = self.__setup_space(physical_space, PhysicalRepresentationSpace)
        self.time_space = self.__setup_space(time_space, TemporalRepresentationSpace)
        
        for space in self.representation_spaces:
            space.attach_dataset(self)
        
    def get_observations(self, observation_type= None, qualities= None, name= None):
        """Accessor to the observations stored in the internal ontology
        
        Parameters:
        - observation_type: the type of the searched observations as defined by the internal ontology
        - qualities: qualities to filter the observations (c.f. `KnowledgeFramework.search`)
        - name: name of the serached observation (c.f. `KnowledgeFramework.search`)"""
        observation_type = observation_type if observation_type is not None else self.knowledge_framework().PointBased_Observation
        return self.knowledge_framework.search(type= observation_type, qualities= qualities, name= name)
    
    def get_unexplained_observations(self, observation_type= None, qualities= None, name= None):
        """Accessor filtering out explained observation
        
        Parameters:
        - observation_type: the type of the searched observations as defined by the internal ontology
        - qualities: qualities to filter the observations (c.f. `KnowledgeFramework.search`)
        - name: name of the serached observation (c.f. `KnowledgeFramework.search`)"""
        observations = self.get_observations(observation_type= observation_type, qualities= qualities, name= name)
        observations = [obs_i for obs_i in observations if len(obs_i.is_Explained_by) == 0]
        return observations
    
    def get_occurrence_observations(self):
        """helper method to access occurrence data, i.e., those having a occurrence quality
        
        :todo: for now the occurrence quality doesn't exist so all the observations are occurrence by default"""
        return self.get_observations(qualities= "occurrence")
    
    def get_orientation_observations(self):
        return self.get_observations(qualities= "dip")
    
    def remove_observation(self, observations, update_extension= True):
        """Removes the given observations stored in this dataset and internal ontology
        
        Parameters:
        - observations: an iterable containing objects of the internal ontology.
        Note that you can use the `search`method to get such a list
        - update_extension: if True (default) the extension will be updated"""
        self.knowledge_framework.remove_all_instances(observations)
        if update_extension: self.update_extension()
            
    def remove_observation_by_name(self, name:str, update_extension= True):
        """Removes the given observations stored in this dataset and internal ontology
        
        Parameters:
        - name: the name of the observation to be removed
        - update_extension: if True (default) the extension will be updated"""
        self.remove_observation(self.get_observations(name= name), update_extension)
        
    def remove_all_observations(self, update_extension= True):
        """Removes all the observations stored in this dataset and internal ontology
        
        Parameters:
        - update_extension: if True (default) the extension will be updated.
        Note: the update is performed only once at the end."""
        self.remove_observation(self.get_observations(), update_extension= False)
        if update_extension: self.update_extension()
        
    def add_observation(self, name: str= None, update_extension= True, **kargs):
        """creates a new observation and adds it to te internal ontology
        
        Parameters:
        - name: the name of the observation (similar to an observation id)
        - update_extension: if True (default) the extension will be updated.
        - **kargs:
          |- any argument whose name corresponds to the `physical_space`coordinate labels or other properties.
          |   Note: all the coordinates must be specified
          |   Also Note: the arguments with None value are removed from the dict"""
        if self.physical_space is  None:
            raise MalissiaBaseError("Trying to add observation while physical space is not set. Please setup_representation_space first")
        
        constructor = self.knowledge_framework.select_object_constructor("Observation", dataset=self, **kargs)
        new_observation = constructor(knowledge_framework= self.knowledge_framework, dataset= self, name= name, **kargs)
        if update_extension: self.update_extension()
        
    def add_occurrence_observation(self, name: str, observed_object:str, occurrence= True, update_extension= True, **kargs):
        """creates a new occurrence observation and adds it to te internal ontology
        
        Parameters:
        - name: the name of the observation (similar to an observation id)
        - observed_object: the name of the observed object
        - occurrence: True (default) if the object was observed here, False if it was observed that it is not there.
        Note that this is different from not having observed that it is here, in which case there should not be an observation.
        - update_extension: if True (default) the extension will be updated.
        - **kargs:
          |- any argument whose name corresponds to the `physical_space`coordinate labels or other properties.
          |   Note: all the coordinates must be specified"""
        if observed_object is not None: kargs["geology"]= observed_object
        if occurrence is not None: kargs["occurrence"]= occurrence
        self.add_observation(name, update_extension= update_extension, **kargs)
    
    def add_orientation_observation(self, name: str, observed_object:str, dip, dip_dir, occurrence= True, update_extension= True, **kargs):
        """creates a new orientation observation and adds it to te internal ontology
        
        Parameters:
        - name: the name of the observation (similar to an observation id)
        - observed_object: the name of the observed object
        - dip: the value of the measured dip (in degrees, 0-90)
        - dip_dir: the value of the dip direction (in degrees, 0-360, from North towards the East)
        - occurrence: True (default) if the object was observed here.
        This is the default behaviour because if the measurement was made here, we assume that the object actually existed
        so this is in itself a proof ox occurrence. However, one might want to record the orientation without specifically attaching any
        observation of occurrence, in which case None should be given for occurrence and the quality won't be set.
        False, would not make much sense as it would imply that the orientation was measured but we observed that the object wasn't there.
        - update_extension: if True (default) the extension will be updated.
        - **kargs:
          |- any argument whose name corresponds to the `physical_space`coordinate labels or other properties.
          |   Note: all the coordinates must be specified"""
        if occurrence is not None: kargs["occurrence"]= occurrence
        if occurrence == False: logging.warning("occurrence parameter was set to False while adding an orientation observation."\
            "This is weird because it would imply the measure was taken but the rock couldn't be observed."\
            "Did you intend to avoid recording the occurrence, in which case you should prefer None isntead of False.")
        
        if observed_object is not None: kargs["geology"]= observed_object
        if (dip is not None) and (dip_dir is not None):
            kargs["dip"]= dip
            kargs["dip_dir"]= dip_dir
        self.add_observation(name, update_extension= update_extension, **kargs)
        
    def __len__(self):
        return len(self.get_observations())
         
    def __str__(self):
        """Description of the dataset
        """
        desc= ["A dataset of type: {:s}".format(type(self).__name__)]
        n = len(self)
        if n == 0:
            desc+= ["- The dataset is empty"]
        else:
            desc+= ["- Size of dataset: {:d}".format(n)]
            desc+= ["- Types of data:"]
            desc+= [" |- occurrence:\t{} entries".format(len(self.get_occurrence_observations()))]
            desc+= [" |- orientation:\t{} entries".format(len(self.get_orientation_observations()))]
        if self.extension is None:
            desc+= ["- Extension: "+str(self.extension)]
        else:
            desc+= ["- Extension:"]
            labels = self.physical_space.coordinate_labels if self.physical_space is not None else ["Coord_{}".format(i) for i in range(3)][:len(self.extension)]
            for dim_i, lim_i in zip(labels,self.extension):
                desc+= [" |- Coord {}: [{}, {}]".format(dim_i, *lim_i)]  
        desc+= ["- Number of spaces this dataset is attached to: {:d}".format(len(self.representation_spaces))]
        if self.physical_space is None:
            desc+= [" |- Physical space: None"]
        else:
            desc+= [" |- Physical space:"+"\n | |".join(self.physical_space.__str__().split("\n"))]
        if self.time_space is None:
            desc+= [" |- Temporal space: None"]
        else:
            desc+= [" |- Temporal space:"+"\n | |".join(self.time_space.__str__().split("\n"))]
        return "\n".join(desc) 
    
    def head(self, n:int= 5):
        """Returns the `n`first data in the dataset"""
        return self.to_dataframe(max_rows=n)
    
    def info(self):
        """Returns a description of the dataset"""
        return self.__str__()
    
    def to_dataframe(self, max_rows:int= None):
        """Creates a `pandas.DataFrame` showing the data in this dataset
        
        Parameters:
        - max_rows: limits the number of rows in the output, unless None is given (default).
        Note: interanlly, all the observations are still recovered from the internal ontology, 
        but only the `max_rows`first ones are show for consision.
        """
        columns = ["name"]
        if self.physical_space is not None:
            columns += self.physical_space.coordinate_labels
        if self.time_space is not None:
            columns += self.time_space.coordinate_labels
        columns += ["dip_dir","dip",'geology', 'occurrence']
        output_frame = pd.DataFrame(columns= columns)
        output_frame.set_index("name",inplace=True)
        
        observations = self.get_observations() 
        observations = observations if max_rows is None else observations[:max_rows]
        for di in observations:
            for prop in di.get_properties():
                if prop.name == '': continue
                if "coord" in prop.name: continue
                output_frame.loc[di.name,prop.name] = prop[di][0]
                
            if self.physical_space is not None:
                for label, val in zip(self.physical_space.coordinate_labels, self.physical_space.get_object_coordinates(di)):
                    output_frame.loc[di.name,label] = val
            if self.time_space is not None:
                for label, val in zip(self.time_space.coordinate_labels, self.time_space.get_object_times(di)):
                    output_frame.loc[di.name,label] = val
        coordinate_types = {label:float for label in self.physical_space.coordinate_labels} if self.physical_space is not None else {}
        time_types = {label:float for label in self.physical_space.coordinate_labels} if self.time_space is not None else {}
        other_types = {'dip_dir':float, 'dip':float, 'geology':str, "occurrence": bool}
        output_frame = output_frame.astype({**coordinate_types, **other_types})
        return output_frame
    
def load_dataset_from_csv(source:str, dataset:GeologicalDataset = None, coordinate_labels = ["X","Y","Z"], **kargs) ->GeologicalDataset:
    """Loads a dataset from a csv file
    
    Parameters:
    - source(str): the source file from which the data should be loaded
    - dataset: the `GeologicalDataset` in which the data will be loaded. If None, the dataset will be created.
    - **arkgs: passed to pandas.read_csv
    
    Return:
    - the `GeologicalDataset` with the newly loaded data (a new `GeologicalDataset` is created if needed)."""
    try:
        dataframe = pd.read_csv(source, **filter_kargs(pd.read_csv,**kargs))
    except Exception as e:
        raise( Exception("This error occurred while loading a dataset from: {}\nAdditional arguments were given: {}".format(source,
                            ",".join(["{}:{}".format(key,val) for key, val in kargs.items()]))))
        
    coordinate_labels = [label for label in coordinate_labels if label in dataframe.columns]
    if len(coordinate_labels) == 0:
        logging.warning("There isn't any coordinate column in the loaded dataset.\nCheck the output and consider changing the separator with sep keyword or cahnge 'coordinate_label' parameter.")
    return load_dataset_from_dataframe(dataframe, dataset, coordinate_labels= coordinate_labels, **filter_kargs(load_dataset_from_dataframe,**kargs) )

def load_dataset_from_dataframe(dataframe, dataset:GeologicalDataset = None, coordinate_labels = ["X","Y","Z"], labels= None, index= None, dtypes= None, clear_existing_data= True):
    """Loads a dataset from a `pandas.DataFrame`
    
    Parameters:
    - dataframe(`pandas.DataFrame`): the source dataframe from which the data should be loaded
    - dataset: the `GeologicalDataset` in which the data will be loaded. If None, the dataset will be created.
    - coordinate_labels: the labels to be used as coordinates of the physical space
    - labels: a dict to relabel the dataframe columns prior to loading in the dataset.
    This is usefull for example when the coordinates in the source aren't labelled the same as in the internal ontology.
    The format is {"old_label":"new_label", ...}.
    - index: the label of the column (in original DataFrame, i.e., before renaming), which is to be used as index
    - dtypes: a dict containing a mapping between column name and type
    - clear_existing_data: if set (default), any oservation stored in the internal ontology is removed prior to loading the new dataset 
    
    Return:
    - the `GeologicalDataset` with the newly loaded data (a new `GeologicalDataset` is created if needed)."""
    if dataset is None:
        physical_space = PhysicalRepresentationSpace(coordinate_labels= coordinate_labels)
        dataset = GeologicalDataset(physical_space = physical_space)
    else:
        if dataset.physical_space is None or clear_existing_data:
            physical_space = PhysicalRepresentationSpace(coordinate_labels= coordinate_labels)
            dataset.setup_representation_space(physical_space = physical_space)
        else:
            if dataset.physical_space.coordinate_labels != coordinate_labels:
                raise MalissiaBaseError("Trying to add data into an existing dataset with different coordiante labels")
                
    if clear_existing_data:
        dataset.remove_all_observations()

    # declare default dtypes here, in case they should be relabelled
    default_coord_types = {key:float for key in dataset.physical_space.coordinate_labels}
    other_types = {'dip_dir':float, 'dip':float, 'geology':str, 'observed_object':str, "occurrence":bool}
    dtypes= dtypes if dtypes is not None else {**default_coord_types, **other_types}
    assert isinstance(dtypes, dict), "dtypes for type management should be given as a dict"
    
    # relabelling
    if labels is not None:
        dataframe = dataframe.rename(columns= labels)
        index= labels[index] if index in labels else index
        dtypes= {labels[key] if key in labels else key: value for key, value in dtypes.items()}
    
    # setting the index
    if index is not None:
        # if already set, reset it
        if type(dataframe.index) != pd.core.indexes.base.Index:
            dataframe = dataframe.reset_index()
        # then set the index
        try:
            dataframe = dataframe.set_index(index)
        except Exception as e:
            raise( Exception("Error while setting the index of the loaded dataset:\nThis is how the dataframe looks like:\n"+dataframe.head().to_string()))
    
    for extra_type_i in dtypes.keys() - set(dataframe.columns):
        del dtypes[extra_type_i]
    dataframe = dataframe.astype(dtypes)
        
    for name_i, values_i in dataframe.iterrows():
        # drop nan values and filter None objects
        values_i = {key:val for key, val in values_i.dropna().items() if val is not None}
        dataset.add_observation(name_i, **values_i, update_extension= False)
    dataset.update_extension()
    
    # resetting the default size when all data are loaded if not specified
    if "size" not in dataframe:
        size = max(physical_space.get_size())
        for di in dataset.get_observations():
            di.size = [float(size/10)]
    
    return dataset

In [ ]:
import inspect

def filter_kargs(target_function,**kargs):
    """Helper function to filter keyword arguments and only pass the needed ones in a function signature"""
    sig = inspect.signature(target_function)
    # check if there is a **kargs in the signature of the function, if yes it is ok as it will take care of the passed extra kargs
    if not any(p.kind == p.VAR_KEYWORD for p in sig.parameters.values()):
        extra_args = kargs.keys() - sig.parameters.keys()
        for args in extra_args:
            del kargs[args]
    return kargs

def debuf_karg_filter(target_function, **kargs):
    print("Kargs filtering:")
    print("before: {",*["{}:{}".format(key,val) for key, val in kargs.items()],"}")
    print("after: {",",".join(["{}:{}".format(key,val) for key, val in filter_kargs(target_function,**kargs).items()]),"}")

In [ ]:
import pandas as pd
filter_kargs(pd.read_csv, **{"sep":";", "truc":"test"})

## Creating a dataset

Data are actually described within the ontology, here thanks to the *Data* class.<br>
Adding new data points calls for creating new *Data* individuals (i.e., instances in the ontology). (see demos below, section datasets)

## Data visualisation

### Testing Data visualisation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def draw_line(center, dip, dir, length= 1, ax= None, color = "black", **kargs):
    ax_plt = plt if ax is None else ax

    center = np.array(center)
    dip_rad = np.deg2rad(dip)
    vec_x =  np.cos(dip_rad)
    if dir == "left": vec_x *= -1
    vec_z = -np.sin(dip_rad)
    vect = 0.5 * length * np.array([vec_x,vec_z])
    start = center - vect
    end = center + vect
    ax_plt.plot([start[0],end[0]],[start[1],end[1]], color = color, **kargs)
    
    return vect
    
def draw_dip_symbol(center, dip, dir, length= 1, polarity= None, ax= None, color = "black", polarity_ratio= 0.4, **kargs):
    ax_plt = plt if ax is None else ax
    
    vect = draw_line(center= center, dip= dip, dir= dir, length= length, ax= ax_plt, color = color, **kargs)
    
    if polarity is not None:
        vect_pol = polarity_ratio * np.array([-vect[1],vect[0]])
        if (dir == "left" and polarity == "up") or (dir == "right" and polarity == "down") : vect_pol *= -1
        ax_plt.arrow(*center,*vect_pol, width=length/100, color = color, **kargs)
        

### Object implementation

In [ ]:
import matplotlib.pyplot as plt
class RenderingObject(object):
    """Class dedicated to rendering a `RepresentationSpace`"""
    registered_drawing_methods = {}
    
    def __init__(self, space:RepresentationSpace):
        """Creating a rendering for the given `RepresentationSpace`"""
        self.space = space

    @classmethod
    def register_drawing_method(cls, object_class_name, drawing_method):
        """Registers a drawing function for the given  `object_class_name`
        """
        if (object_class_name is None):
            raise MalissiaBaseError("Registering drawing method for an undefined object class.")
        if drawing_method is None: raise MalissiaBaseError("Registering  an undefined drawing method for an object class.")
        
        cls.registered_drawing_methods[object_class_name] = drawing_method
        
    @classmethod
    def get_drawing_method(cls, object_class):
        """Accessor to the drawing method for a given object
        
        Parameters:
        - object_class_name (str): is the name of the class of the object to be drawned
        """
        if (object_class is None):
            raise MalissiaBaseError("Drawing methods are registered by object type. None was given, please give in a valid class.")
        if object_class not in cls.registered_drawing_methods:
            raise MalissiaBaseError("No drawing method registered for this class: "+ object_class)
        return cls.registered_drawing_methods[object_class]
        
    def setup_ax(self, ax= None):
            if ax is not None:
                self.plt_ax = ax
            elif self.plt_ax is None:
                self.plt_ax = plt
                
    def draw_interpreted_objects(self, ax= None, setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        for object_i in self.space.get_interpreted_objects():
            object_class_name = type(object_i).name
            drawing_method = self.get_drawing_method(object_class_name)
            drawing_method(object_i, representation_space= self, ax= ax, setup_drawing= False, **kargs)
                
class AxisAlignedCrossSection(RenderingObject):
    """A specialised `Rendering` that procudes a cross-section.
    
    The cross-section is defined by two coordinates of the rendered `PhysicalRepresentationSpace`"""
    registered_drawing_methods = {}
    
    def __init__(self, space:PhysicalRepresentationSpace, u= None, v= None, ax= None):
        """Creates a cross section through the given physical space
        
        Parameters:
        - space: the space through which the cross section is going.
        Note: the given space must be a `PhysicalRepresentationSpace` and have two or more coordinates
        - u: the label of the abscissa axis among the physical space coordinates.
        By default, if None is given, the first axis of the space is used.
        - v: the label of the ordinate axis among the physical space coordinates.
        By default, if None is given, the last axis of the space is used,
        effectively using both coordinates if the space is 2D, but a vertical cross-section if it is 3D.
        - ax: the matplotlib axis in which the space is to be rendered.
        If None (default), then a new axis will be created."""
        assert isinstance(space,PhysicalRepresentationSpace), "The given representation space must be a PhysicalRepresentationSpace, here: "+str(type(space))
        assert space.dimension > 1, "Cross sections are only possible through spaces of dimensions >=2, here: "+str(space.dimension)
        self.space = space
        
        if u is None:
            self.u = self.space.coordinate_labels[0]
            self.u_index = 0
        else:
            assert u in self.space.coordinate_labels, "The first given coordinate (u={}) is not in the represented space ({})".format(u, ",".join(space.coordinate_labels))
            self.u = u
            self.u_index = np.argwhere(self.space.coordinate_labels == self.u)[0,0]
        
        if v is None:
            self.v = space.coordinate_labels[-1]
            self.v_index = len(space.coordinate_labels) - 1
        else:
            assert v in space.coordinate_labels, "The second given coordinate (v={}) is not in the represented space ({})".format(v, ",".join(space.coordinate_labels))
            self.v = v
            self.v_index = np.argwhere(self.space.coordinate_labels == self.v)[0,0]
            
        self.plt_ax = None
                
    def setup_drawing(self, ax= None):
            self.setup_ax(ax)
            
            ax = self.plt_ax.gca() if self.plt_ax == plt else self.plt_ax
            ax.set_aspect("equal")
            ax.set_xlim( *self.space.extension[self.u_index])
            ax.set_xlabel(self.u)
            ax.set_ylim( *self.space.extension[self.v_index])
            ax.set_ylabel(self.v)
    
    def draw_line(self, center, dip, dir, length= 1, color = "black", setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        center = np.array(center)
        dip_rad = np.deg2rad(dip)
        vec_x =  np.cos(dip_rad)
        if dir == "left": vec_x *= -1
        vec_z = -np.sin(dip_rad)
        vect = 0.5 * length * np.array([vec_x,vec_z])
        start = center - vect
        end = center + vect
        self.plt_ax.plot([start[0],end[0]],[start[1],end[1]], color = color, **kargs)
        
        return vect        
    
    def draw_dip_symbol(self, di, length= None, polarity= None, color = "black", polarity_ratio= 0.4, setup_drawing= True, ax= None, zorder= 20, **kargs):
        if setup_drawing: self.setup_drawing(ax)

        dip = di.dip[0]
        dir = "right" if di.dip_dir[0] < 180 else "left"
        center = self.get_center_coordinates(di)
        
        length = 0.5 * di.size[0] if di.size is not None else length
        if length is None or length is np.nan:
            length = np.abs(np.max(self.space.extension[:,1] - self.space.extension[:,0])) / 20
        
        vect = self.draw_line(center= center, dip= dip, dir= dir, length= length, color = color, setup_drawing=False, ax=ax, zorder= zorder, **kargs)
        
        if di.polarity is not None:
            polarity = "up" if di.polarity[0] else "down"
        if polarity is not None:
            vect_pol = polarity_ratio * np.array([-vect[1],vect[0]])
            if (dir == "left" and polarity == "up") or (dir == "right" and polarity == "down") : vect_pol *= -1
            self.plt_ax.arrow(*center,*vect_pol, width=length/100, color = color,  zorder= zorder, **kargs)
        
    def filter_section_coordinates(self, coord):
        return coord[[self.u_index,self.v_index]]
        
    def get_center_coordinates(self,di):
        center = di.has_Center[0]
        coord = self.space.get_object_coordinates(center)
        return self.filter_section_coordinates(coord)
    
    def get_corner_coordinates(self, surf):
        corners = surf.has_Representation
        coord = np.array([self.space.get_object_coordinates(corner_i) for corner_i in corners]).T
        return self.filter_section_coordinates(coord)
    
    def draw_dip_data(self, ax= None, polarity= "up", setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        for dataset_i in self.space.get_datasets():
            for di in dataset_i.get_orientation_observations():
                self.draw_dip_symbol(di, setup_drawing= False, ax=ax, **kargs) 
    
    def draw_point(self, center, color = "black", marker="*", edge_color= "black", zorder= 10, setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        center = self.get_center_coordinates(center)
        self.plt_ax.scatter(*center, color = color, marker= marker, edgecolors= edge_color, zorder=zorder, **kargs)
        
    def draw_plane(self, plane, color = "limegreen", marker=".", edge_color= "black", zorder= 2, setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        corner_coords = self.get_corner_coordinates(plane)
        corner_coords = np.append(corner_coords,corner_coords[:,0].reshape((2,1)),axis=1)
        self.plt_ax.scatter(*corner_coords, color = color, marker= marker, edgecolors= edge_color, zorder=zorder, **kargs)
        self.plt_ax.plot(*corner_coords, color = color, marker= None, zorder=zorder-1, **kargs)
    
    def draw_occurrence_symbol(self, d, color = "lightblue", marker="*", edge_color= "black", zorder= 10, setup_drawing= True, ax= None, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        self.draw_point( d, color, marker, edge_color= edge_color, zorder=zorder, setup_drawing= False, ax=ax, **kargs) 
        
    def draw_occurrence_data(self, ax= None, setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        
        for dataset_i in self.space.get_datasets():
            for di in dataset_i.get_occurrence_observations():
                self.draw_occurrence_symbol(di, setup_drawing= False, ax=ax, **kargs) 
        
    def show(self, ax= None, setup_drawing= True, **kargs):
        if setup_drawing: self.setup_drawing(ax)
        self.draw_occurrence_data(ax=ax, setup_drawing= False, **kargs)
        self.draw_dip_data(ax=ax, setup_drawing= False, **kargs)

#### Drawing methods

In [ ]:
def draw_observation_in_AxisAlignedCrossSection(
        observation,
        representation_space,
        ax= None,
        setup_drawing= True,
        **kargs
    ):
    if setup_drawing:
        representation_space.setup_drawing(ax)
    else:
        representation_space.setup_ax(ax)
        
    if observation.occurrence is not None:
        representation_space.draw_occurrence_symbol(observation, ax= ax, setup_drawing= False, **kargs)
    if observation.dip is not None:
        representation_space.draw_dip_symbol(observation, ax= ax, setup_drawing= False, **kargs)
        
if mogi().PointBased_Observation in AxisAlignedCrossSection.registered_drawing_methods:
    del AxisAlignedCrossSection.registered_drawing_methods[mogi().PointBased_Observation]
AxisAlignedCrossSection.register_drawing_method(mogi().PointBased_Observation, draw_observation_in_AxisAlignedCrossSection)

In [ ]:
def draw_surface_part_in_AxisAlignedCrossSection(
        strati_part,
        representation_space,
        ax= None,
        setup_drawing= True,
        **kargs
    ):
    if setup_drawing:
        representation_space.setup_drawing(ax)
    else:
        representation_space.setup_ax(ax)
        
    plane = strati_part.has_Representation[0]
    representation_space.draw_plane(plane, **kargs)
    
        
if mogi().Stratigraphic_Part in AxisAlignedCrossSection.registered_drawing_methods:
    del AxisAlignedCrossSection.registered_drawing_methods[mogi().Stratigraphic_Part]
AxisAlignedCrossSection.register_drawing_method(mogi().Stratigraphic_Part, draw_surface_part_in_AxisAlignedCrossSection)

In [ ]:
def draw_situation_in_AxisAlignedCrossSection(
        situation,
        representation_space,
        ax= None,
        setup_drawing= True,
        **kargs
    ):
    if setup_drawing:
        representation_space.setup_drawing(ax)
    else:
        representation_space.setup_ax(ax)
        
    for feature_i in situation.features:
        drawing_function = representation_space.get_drawing_method(feature_i.is_instance_of[0])
        drawing_function(feature_i, representation_space, color= "deepskyblue", ax = ax, setup_drawing= False, **kargs)
        
if InterpretationSituation in AxisAlignedCrossSection.registered_drawing_methods:
    del AxisAlignedCrossSection.registered_drawing_methods[InterpretationSituation]
AxisAlignedCrossSection.register_drawing_method(InterpretationSituation, draw_situation_in_AxisAlignedCrossSection)

### Tasks

The tasks in the algorithm of interpretation are not directly specified in the interpretation process but are made abstract to make it easier to implement alternative ways to proceed.

Our implementation separates two aspects of the problem:
1. **Task**: providing an interface for describing the task to be achieved
2. **Strategy**: providing the implementation for achieving the task.

Both are represented by a separate interface; only derived classes of Task that have a defined strategy are actually applicable.

There is a weak distinction between two kind of tasks:
- Strategies: for implementing parts of the overall algorithm that are fully dependent on the user's choice. In other terms, 
there is no obvious or ultimately best strategy for achieving a task, just actions that can be carried out in different ways.
- Heuristics: for providing quick and/or easy approximate solutions to a given problem. Here, there is in theory a correct answer
but it might be to difficult or expansive to infer it



In [ ]:

import sys
from enum import Enum, Flag, auto
import random, string
    
class MissingTaskImplementationError(MalissiaBaseError):
    """Exception generated when trying to run a task for which no implementation was selected."""
    
    def __init__(self):
        super().__init__ ("This error occurred because a task has been executed without a proper implementation defined by a strategy.\n"\
            "Make sure to use fully implemented Task class (ie. not the base ones) and that self.strategy is set")
        

class StrategyType(Flag):
    """Defines flags for specifying behaviour of the Strategy.
    
    DEFAULT: no specific behaviour
    USER: it will ask user for some input
    RANDOM: it sill set some results in a random way
    BRUTE_FORCE: it will try all options and select the best result"""
    DEFAULT = 0
    USER = auto()
    RANDOM = auto()
    BRUTE_FORCE = auto()

class Strategy(object):
    """Strategies are pieces of algorithm that are implementing a specific task.
    
    Strategy provides the generic interface of algorithms implementing a `Task`
    and determines which tasks are actually concrete. In other words, a `Task` that does not inherit from `Strategy`
    can not be executed.
    
    `Strategy` provides the following interface (which has to be implemented in inheriting classes):
    - class attributes:
      - name: the human-readable name of the strategy
      - short_desc: a one-line description of the strategy
      - full_desc: a complete description of the strategy
    - attributes initialised by __init__:
      - strategy_type: the type of strategy (cf. `StrategyType`)
    - methods:
      - check_applicability(): which checks if the algorithm can be applied in the context defined by the task
      - execute(): which effectively runs the algorithm
    
    Strategies differ from Heuristics, in that the implemented task does not have an obvious/natural/true result or way to proceed,
    but it rather a matter of choice, hence a strategy."""
    
    name = "Strategy"
    short_desc = "Generic Abstract Strategy"
    full_desc = "This is not supposed to be implemented."
    strategy_type= StrategyType.DEFAULT
    
    def __init__(self, type= None, **kargs):
        """Initialises common attributes for the `Strategy` class
        
        Parameters:
        - type (`StrategyType`): the type of strategy implement, mostly to specify if user defined or random"""
        if type is not None:
            self.strategy_type = type
        
    def check_applicability(task = None):
        """Checks if this algorithm can be applied in the current context
        
        This base class method actually triggers an Exception to make sure the methods is implemented in inheriting class"""
        raise MalissiaBaseError("Trying to check applicability of the {} strategy but this class doesn't implement "\
            "the check_applicability method or run Strategy.check_applicability() instead.".format(__class__.__name__))
        
    def execute(self, task):
        """runs the algorithm
        
        This base class method actually triggers an Exception to make sure the methods is implemented in inheriting class.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        raise MalissiaBaseError("Trying to execute the {} strategy but this class doesn't implement the execute method "\
            "or run Strategy.execute() instead.".format(self.__class__.__name__))

class UserInputStrategy(Strategy):
    """Strategy based on interaction with the user."""
    name = "UserInputStrategy"
    short_desc = "Generic User Strategy"
    full_desc = "This is implementing generic user interaction."
    strategy_type= StrategyType.USER
    
    def __init__(self, **kargs):
        """Initialises a User based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task):
        """Checks if user interaction is possible (based on sys.ps1)
        
        This kind of algorithm is also restricted to result tasks because actions would be too complex to ask to users.
        """
        applicable = isinstance(task, ResultTask) 
        applicable &= UserInputStrategy.check_user_interface()
        return applicable
    
    def check_user_interface():
        return hasattr(sys, "ps1")
    
    def execute(self, task):
        """Asks user for the result to give.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        task.result = task.result_type(input("Please give in an input that can be used as a '{}':".format(task.result_type.__name__)))
    
class RandomStrategy(Strategy):
    """Strategy based on random generator."""
    name = "RandomStrategy"
    short_desc = "Generic Random Strategy"
    full_desc = "This is implementing generic random generator."
    strategy_type= StrategyType.RANDOM
    
    supported_types = (str, float, int)
    
    def __init__(self, **kargs):
        """Initialises a Random based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task):
        """Checks if applicable
        
        i.e., for a ResultTask and result_type being in supported_types
        """
        applicable = isinstance(task, ResultTask)
        applicable &= task.result_type in __class__.supported_types
        return applicable
    
    def execute(self, task):
        """apply the strategy.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        if task.result_type == str:
            task.result = "".join(random.choices(string.ascii_lowercase, k=10))
        elif task.result_type == float:
            task.result = random.random()
        elif task.result_type == int:
            task.result = random.randint(0,10)
        else:
            raise MalissiaBaseError("Unsupported type for RandomStrategy. Request: {}, Available: {}.".format(task.result_type,", ".join(__class__.supported_types)))
        
class Task(object):
    """Task is an abstract base class to provide the generic interface for any tasks of the `GeologicalInterpretationProcess`
    
    Task provides the following interface (which has to be implemented in inheriting classes):
    - class attributes:
      - name: the human-readable name of the task
      - short_desc: a one-line description of the task
      - full_desc: a complete description of the task
      - available_strategies: a list of available strategy types for implementing the `Task`.
        NB: strategies are related to `Task` by composition to make it possible to derive the tasks and still inherit from the base class strategies.
    - attributes initialised by __init__:
      - context: a context in which the task must be executed, typically the `GeologicalInterpretationProcess` instance
      - strategy: the selected strategy instance for implementing the task if any.
    """
    
    name = "Task"
    short_desc = "Generic Abstract Task"
    full_desc = "This is not supposed to be implemented."
    available_strategies = set()
    
    def __init__(self, context, **kargs):
        """Initialises common attributes for the `Task` class
        
        Parameters:
        - task_name: the human-readable name of the task
        - task_short_desc: a one-line description of the task
        - task_full_desc: a complete description of the task
        - context: the context of the task, this should provide the adapted interface for giving the required information
        """
        self.context = context

    def set_strategy(self, strategy):
        """Setter for the strategy, this ensures everything is set properly"""
        assert not ( (strategy is not None) and (not isinstance(strategy, Strategy))), "strategy must be either None or an instance of a Strategy (not a class)"
        self.strategy = strategy
        
    def check_applicability(self):
        """Checks that the task can be executed (ie. a strategy is set and applicable)"""
        if self.strategy is None: return False
        return self.strategy.__class__.check_applicability(self)
    
    def execute(self, bypass_applicability_check = False):
        """Method to run the selected implementation of the task.
        
        If no strategy is implemented/selected, this triggers an exception (`MissingTaskImplementationError`).
        
        parameters:
        - bypass_applicability_check (bool): if True, the applicability of the task won't be checked beforehand. Default: False"""
        if (not hasattr(self, "strategy")) or (self.strategy is None):
            raise MissingTaskImplementationError()
        elif (not bypass_applicability_check) and (not self.check_applicability()):
            raise MalissiaBaseError("executed a Task with a strategy that was not applicable in the current context.")
        else: self.strategy.execute(self)
        
class ActionTask(Task): 
    """Task in charge of applying some actions.
    
    Such tasks do not typically store a result but instead interact and possibly modify the context.
    """
    name = "ActionTask"
    short_desc = "Generic Abstract Task for actions"
    full_desc = "This is a generic action, meaning it doesn't hold a result but instead affects the context directly. This is not supposed to be implemented directly."
    available_strategies = Task.available_strategies
    
    def __init__(self, context, **kargs):
        """Initialises an ActionTask
        
        Parameter:
        - context: provides a context with the appropriate (task specific) interface. This parameter is compulsory."""
        super().__init__(context= context, **kargs)
        
class ResultTask(Task):
    """`Task` in charge of creating a result.
    
    Such tasks should not modify the context but only generate a result stored the `self.result` attribute.
    
    `UserInputStrategy` and `RandomStrategy` are automaticall added as available strategies
    """
    name = "ResultTask"
    short_desc = "Generic Task for generating results"
    full_desc = "This is a generic result generator, meaning it doesn't affect the context but holds a result instead."
    available_strategies = Task.available_strategies | set((UserInputStrategy, RandomStrategy))
    
    def __init__(self, result_type, context = None, **kargs):
        """Initialises a ResultTask
                
        Additional parameters (for others see `Task`):
        - result_type: the type of expeted result
        - context: provides a context with the appropriate (task specific) interface. This parameter is compulsory."""
        super().__init__(context= context, **kargs)
        self.result_type = result_type
        self.result = None

    def execute(self, bypass_applicability_check = False, return_result= True):
        """Additional behaviour for `ResultTask`
        
        Runs the `Task.execute()` then return result if the option is activated (default)"""
        super().execute(bypass_applicability_check= bypass_applicability_check)
        if return_result: return self.result
        
class StrategyFactory(object):
    """This class gathers all the strategies available for the `GeologicalInterpretationProcess`.
    
    The strategies are automatically recovered from the subclasses of Task and Strategy.
    Three dictionaries are registering the task and corresponding strategies:
    - self.tasks: are the Defined Tasks having defined possible Strategies
    - self.specified_tasks: are the tasks with a specific available strategy (i.e., only one)
    - self.implemented_tasks: tasks that have an implemented strategy
    NB: they aren't used for generating the tasks, but just for reporting the existing tasks.
    """
    
    def __init__(self):
        """Initialisation of the strategy storage."""
        self.tasks = {}
        self.specified_tasks = {}
        self.implemented_tasks = {}
        self.tasks_without_strategy = {}
        self.list_available_tasks()
        self.list_specified_tasks()
        self.list_implemented_tasks()
        self.list_tasks_without_strategy()
        
    def generate_task(self, task_type, strategy_type= None, strategy_class= None, context= None, **kargs):
        """Creates a task of a given type and instanciate appropriate strategy
        
        Parameters:
        - task_type: the class of the task to be created
        - strategy_type: the type of strategy to be used, see `StrategyType`
        - strategy_class: to force the strategy selection, its class can be directly given here
        - context: a context that can be passed to the task to define it (typically the `GeologicalInterpretationProcess`)
        - kargs: keyword arguments that can be passed to the task initialisation"""
        
        # Create the task
        task = task_type(context = context, **kargs)
        
        if strategy_class is None:
            # list strategies
            strategies = np.array(list(task_type.available_strategies))
            
            # check applicability to the task and context
            applicable_strategy = [candidate_strategy for candidate_strategy in strategies if candidate_strategy.check_applicability(task)]
            
            # filter strategy types
            if strategy_type is not None:
                applicable_strategy = [candidate_strategy for candidate_strategy in applicable_strategy if candidate_strategy.strategy_type == strategy_type]
                
            if len(applicable_strategy) == 0:
                message = ["No implementation of the requested task ({}) found in this context.".format(task_type)]
                message += ["Note that the following strategies were available:\n"+"\n".join(strategies)]
                message += ["Note that the following strategies were applicable:\n"+"\n".join(applicable_strategy)]
                message += ["Note that the following strategies were filtered out:\n"+"\n".join([candidate_strategy for candidate_strategy in applicable_strategy if candidate_strategy.strategy_type != strategy_type])]
                raise MalissiaBaseError("\n".join(message))
             
            # this could be defined by a strategy as well, e.g. pick first of random
            strategy_class = random.choice(applicable_strategy)
            
        else:
            if strategy_class.check_applicability(task) == False:
                raise MalissiaBaseError("The proposed implementation ({}) of the requested task ({}) is not applicable in this context.".format(strategy_class,task_type))
                
        # instanciate the strategy
        strategy_instance = strategy_class(**kargs)
        task.set_strategy(strategy_instance)
            
        return task
    
    def list_available_tasks(self):
        """List the Tasks that are fully defined (ie. having attached strategies)"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            strategies = task_i.available_strategies
            if len(strategies) > 0:
                self.tasks[task_i] = strategies 
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
    
    def list_specified_tasks(self):
        """List the Tasks that have only one possible strategy"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            strategies = task_i.available_strategies
            if len(strategies) == 1:
                self.specified_tasks[task_i] = list(strategies)[0]
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
    
    def list_implemented_tasks(self):
        """List the Tasks that are implemented (ie. having an attached strategy instance)"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            if hasattr(task_i,"strategy") and task_i.strategy is not None:
                self.implemented_tasks[task_i] = task_i.strategy 
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
            
    def list_tasks_without_strategy(self):
        """Lists the tasks that have no strategies nor subclass"""
        task_pile = list(Task.__subclasses__())
        while len(task_pile) > 0:
            task_i = task_pile.pop()
            
            strategies = task_i.available_strategies
            if (len(strategies) == 0) and (len(task_i.__subclasses__()) == 0):
                self.tasks_without_strategy[task_i] = ()
            
            sub_tasks = list(task_i.__subclasses__())
            task_pile += sub_tasks
    
    def __str__(self):
        """Describe the stored strategies"""
        self.list_available_tasks()
        self.list_specified_tasks()
        self.list_implemented_tasks()
        self.list_tasks_without_strategy()
        
        desc = ["Strategy Factory:"]
        desc+= ["- Available Tasks:"]
        for task, strat_list in self.tasks.items():
            desc += ["  |- {}:".format(task.name)]
            if len(strat_list) == 0:
                desc += ["  |[Warning]: no strategy provided for this task"] 
            for strat_i in strat_list:
                desc += ["  |  |- {}{}: {}".format(strat_i.__class__.__name__, "" if strat_i.strategy_type is StrategyType.DEFAULT else "["+strat_i.strategy_type.name+"]", strat_i.short_desc)]
        
        if len(self.specified_tasks) > 0:
            desc+= ["- Specified Tasks:"]
            for task, strat in self.specified_tasks.items():
                desc += ["  |- {}: {}{}({})".format(task.name, strat.name, "" if strat.strategy_type is StrategyType.DEFAULT else "["+strat.strategy_type.name+"]", strat.short_desc)]
        if len(self.implemented_tasks) > 0:
            desc+= ["- Implemented Tasks:"]
            for task, strat in self.implemented_tasks.items():
                desc += ["  |- {}: {}{}({})".format(task.name, strat.name, "" if strat.strategy_type is StrategyType.DEFAULT else "["+strat.strategy_type.name+"]", strat.short_desc)]
        if len(self.tasks_without_strategy) > 0:
            desc+= ["- Tasks without strategy:"]
            for task, strat in self.tasks_without_strategy.items():
                desc += ["  |- {}: [Warning]: no strategy provided for this task".format(task.name)]
        return "\n".join(desc)
        
    
__default_strategy_factory__ = StrategyFactory()
    
class HeuristicsFactory(object):
    """This class provides Heuristics for the `GeologicalInterpretationProcess`.
    
    Heuristics are pieces of algorithm that are implementing a specific computational or decisionnal task.
    They differ from strategies, in that there exists a true result even if it is not know or two complex to infer accurately.
    Heuristics try to provide acceptable approximate results with a simple algorithms to save either computation time, memory, or complexity.

    The heuristics are stored in a dictionnary (`self.__heuristics`) that associates tasks with a list of possible implementation. 
    """
    
    def __init__(self):
        """Initialises the Heuristics storage."""
        self.__heuristics = {}
        
# print(__default_strategy_factory__)

In [ ]:
class ExampleTask(ResultTask):
    """class in charge of showing how to implement new tasks
    
    Such task is independent of a context and ignores it if passed."""
    
    name = "ExampleTask"
    short_desc = "gives an example"
    full_desc = "Demonstrates the implementation of new tasks."
    def __init__(self, result_type= None, **kargs):
        """example of initialisation of a result task
        
        Parameters:
        - result_type: the type of expeted result. If None, one of the RandomStrategy supported type is used."""
        result_type = result_type if result_type is not None else random.choice(RandomStrategy.supported_types)
        super().__init__(result_type= result_type)
        
class UserExampleTask(ExampleTask):

    name = "UserExampleTask"
    short_desc = "example user strategy"
    full_desc = "Gives an example of implementation of user defined algorithm."
    available_strategies = {UserInputStrategy}
    
    def __init__(self, result_type= str, **kargs):
        super().__init__(result_type= result_type)
        strategy = UserInputStrategy()
        super().set_strategy(strategy)
    
class RandomExampleTask(ExampleTask):

    name = "RandomExampleTask"
    short_desc = "example random strategy"
    full_desc = "Gives an example of implementation of random defined algorithm."
    available_strategies = {RandomStrategy}
    
    def __init__(self, result_type= str, **kargs):
        super().__init__(result_type= result_type)
        strategy = RandomStrategy()
        super().set_strategy(strategy)
        
# print(__default_strategy_factory__)


### Specific Tasks implementation

In [ ]:
class UserSelectStrategy(UserInputStrategy):
    """Strategy based on interaction with the user."""
    name = "UserSelectStrategy"
    short_desc = "Selection User Strategy"
    full_desc = "This is implementing selection based on user interaction."
    type= StrategyType.USER
    
    def __init__(self, **kargs):
        """Initialises a User based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task, context = None):
        """Checks if applicable
        
        uses UserInputStrategy.check_applicability and SelectionTask type
        """
        applicable = isinstance(task, SelectionTask) 
        applicable &= UserInputStrategy.check_user_interface()
        return applicable
    
    def execute(self, task):
        """Asks user for the result to give.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        message = "Please select an option amongst the following (give its index):\n "+"\n".join(["{}: {}".format(i,val) for i, val in enumerate(task.choices)])
        try:
            selected = int(input(message))
            task.result = task.result_type(task.choices[selected])
        except ValueError:
            raise MalissiaBaseError("User input must be in the form of an integer corresponding to the selected index.")
        except IndexError:
            raise MalissiaBaseError("The selected index must be within [{}, {}].".format(0,len(task.choices)-1))
        return task.result
    
class RandomSelectStrategy(RandomStrategy):
    """Strategy based on random generator."""
    name = "RandomSelectStrategy"
    short_desc = "Random Selection Strategy"
    full_desc = "This is implementing a random selection."
    type= StrategyType.RANDOM
    
    def __init__(self, **kargs):
        """Initialises a Random based strategy"""
        super().__init__(**kargs)
        
    def check_applicability(task, context = None):
        """Checks if applicable
        
        Needs to be set for a SelectionTask with at least one choice
        """
        applicable = isinstance(task, SelectionTask)
        applicable &= len(task.choices) >0
        return applicable
    
    def execute(self, task):
        """perform random selection
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        
        if len(task.n) == 1:
            if task.n[0] == 1:
                task.result = random.choice(task.choices)
                return task.result_type(task.result)
            else:
                k = task.n
        else:
            k = np.cumprod(task.n)[-1]
            
        task.result = random.choices(task.choices, k = k)
        if len(task.n) > 1:
            task.result.reshape(task.n)
            
        return task.result.astype(task.result_type)

class SelectionTask(ResultTask):
    """SelectionTask performs a selection among a series of possible results."""
    
    name = "SelectionTask"
    short_desc = "makes a selection"
    full_desc = "Performs a selection among a series of possible results."
    available_strategies = set((UserSelectStrategy, RandomSelectStrategy))
    
    def __init__(self, choices, n= 1, result_type= None, context= None, **kargs):
        """Initialisation of a selection task
        
        Parameters:
        - choices: a series of possible values for the result. Will be transformed into a numpy array and flatten (np.array(choices).ravel())
        - n (int): number of elements to be picked, alternatively if a shape is given, the shape of the expected result array
        - result_type: the type of expeted result. If None, the type of the choices will be used by default."""
        self.choices = np.array(choices).ravel()
        self.n = np.array(n).ravel()
        result_type = result_type if result_type is not None else self.choices.dtype.type
        super().__init__(result_type= result_type)
        


In [ ]:
class SelectObservation(SelectionTask):
    """Selection of an observation"""
    name = "SelectObservation"
    short_desc = "selects observations"
    full_desc = "Performs a selection among possible observations."
    
    def __init__(self, n= 1, choices = None, dataset= None, context= None, **kargs):
        """Initialisation of a selection task
        
        Parameters:
        - choices: a series of possible observations. If None (default), the dataset in the context will be used (and therefore can't be None)
        - dataset: a GeologicalDataset to bring the available observations unless already defined by choices 
        - context: if given a `GeologicalInterpretationProcess`, will be used to define the possible selection unless, choices or dataset are given.
        - n (int): number of elements to be picked, alternatively if a shape is given, the shape of the expected result array"""
        choices = choices if choices is not None else self.get_choices(dataset= dataset, context= context, **kargs)
        super().__init__(choices = choices, n= n, context = context, **kargs)

    def get_choices(self, choices = None, dataset:GeologicalDataset= None, context= None, **kargs):
        """generates the choices list from different possible sources
        
        Parameters:
        - choices: a series of possible observations. If None (default), the dataset in the context will be used (and therefore can't be None)
        - dataset: a GeologicalDataset to bring the available observations unless already defined by choices 
        - context: if given a `GeologicalInterpretationProcess`, will be used to define the possible selection unless, choices or dataset are given.
        """
        if (context is None) and (dataset is None):
            raise MalissiaBaseError("Either a series of choices, a dataset, or a context must be given.")
        if dataset is None:
            dataset = context.dataset
        choices = dataset.get_observations()
        return choices

In [ ]:
SelectObservation.available_strategies

## Interpretation Workflow

The interpretation process in itself is run in a **GeologicalInterpretationProcess** and follow a very simple and generic algorithm.<br>
This algorithm implements a Deming wheel process of continual improvement:
1. Plan:
    1. Select a situation
    2. Select an action
2. Do: Implement the action (e.g., CreateInterpretationElement)
    1. List features
    2. Identify possible explanations
    3. Rank/chose explanations
    4. Instanciate individuals
    5. Infer and set parameters
3. Check: Evaluate consistency
    1. Evaluate internal consistency
    2. Evaluate relational likelihood
    3. Evaluate feature explanation
4. Act: Generate anomalies and report

In [ ]:
import random
from enum import Enum, Flag, auto
class TerminationFlag(Flag):
    """Defines flags for specifying why the `GeologicalInterpretationProcess` stoped
    
    DEFAULT: it has not been terminated
    USER: was terminated by user
    MAX_ITER: maximum iteration number reached
    """
    DEFAULT = 0
    USER = auto()
    MAX_ITER = auto()
   

def select_user_unexplained_feature(process):
   """user selects a single feature to be explained"""
   observations = process.dataset.get_unexplained_observations()
   message = "; ".join(["{}: {}".format(i,obs_i.name) for i, obs_i in enumerate(observations)])
   selected_feature_index = int(input(message))
   selected_feature = observations[selected_feature_index]
   process.situation = InterpretationSituation( features= [selected_feature], process= process)
   
def select_single_random_feature(process):
   """randomly selects a single feature to be explained"""
   observations = process.dataset.get_observations()
   selected_feature = random.choice(observations) if len(observations) > 0 else []
   process.situation = InterpretationSituation( features= [selected_feature], process= process)
   
def select_single_random_unexplained_feature(process):
   """randomly selects a single feature to be explained"""
   observations = process.dataset.get_unexplained_observations()
   selected_feature = random.choice(observations) if len(observations) > 0 else []
   process.situation = InterpretationSituation( features= [selected_feature], process= process)

def create_interpretation(process):
   pass

def select_interpretation_move():
   pass

class GeologicalInterpretationProcess(object):
   """GeologicalInterpretationProcess implements the core process of a geological intepretation.
    
    It connects all the required elements and resulting artefacts relatively to a given interpretation sequence:
     - knowledge_framework: a GeologicalKnowledgeFramework
     - dataset: a GeologicalDataSet that interfaces all the available data
     - representation_space: a `RepresentationSpace` defining the study zone
     - strategies: a dict that associates algorithm stages with preferred options"""
   
   default_strategies = {
      "SituationSelection": select_single_random_unexplained_feature,
      "InterpretationMoveSelection": select_interpretation_move
   }
     
   def __init__(self,
                dataset: GeologicalDataset,
                representation_space: RepresentationSpace = None,
                knowledge_framework= None,
                strategies = {}):
      """Creates a GeologicalInterpretationProcess
        
      ---------------------------
      Parameters:
       - dataset (GeologicalDataset): a dataset to be explained by this interpretor
       - representation_space (RepresentationSpace): the representation space that must be explainined by the interpretation.
       If None (default), the physical space of the dataset is used instead.
       - knowledge_framework: a GeologicalKnowledgeFramework that defines the concepts used for this interpretation.
         If None is given, the the default knowledge framework is used (`GeologicalKnowledgeManager().get_knowledge_framework()`)
       - strategies: a dict that associates algorithm stages with preferred options
      """
      self.dataset = dataset
      self._interpreted_objects = set()
      self.representation_space = representation_space if representation_space is not None else self.dataset.physical_space
      self.knowledge_framework= GeologicalKnowledgeManager().get_knowledge_framework() if knowledge_framework is None else knowledge_framework
      
      self.strategies = {**GeologicalInterpretationProcess.default_strategies, **strategies}
      
      self.update_status()
         
      self.interpretation_moves ={
         "NewInterpretation": self.apply_new_interpretation,
         "UpdateInterpretation": self.apply_update_interpretation,
         "RemoveInterpretation": self.apply_remove_interpretation
      }
      
   def __init_status(self, init):
      """creates the status on first run or reset"""
      if not hasattr(self, "status"):
         self.status = {}
         init = True
      if init:
         self.status["epoch"] = None # index of the current iteration
      
   def update_status(self, init= False):
      """Performs status update operations
      
      If the status is not set yet, this method will also create it and initialise it.
      
      Parameters:
      - init (Bool): if True, the status will be reset to initial values. Default is False."""
      self.__init_status(init)
            
      self.status["coverage"] = self.evaluate_coverage() # ratio of representation space covered by an explanation
      self.status["data_explanation_ratio"] = self.evaluate_data() # ratio of explained observations
      self.status["anomaly_explanation_ratio"] =  self.evaluate_anomalies() # ratio of explained observations
      self.status["termination_criterion"] = TerminationFlag.DEFAULT # gives explanation about why it terminated
      
   def register_interpreted_object(self, object, **kargs):
      """Registers an interpreted object"""
      self.registered_interpreted_objects.add(object)
      self.physical_space.attach_interpreted_object(object, **kargs)
      
   def evaluate_coverage(self):
      """Evaluates the amount of physical space that is explained
      
      Returns:
      - a float between 0 and 1, representing the ratio of covered space"""
      return 0
      
   def evaluate_data(self):
      """Evaluates the amount of data that is explained
      
      Returns:
      - a float between 0 and 1, representing the ratio of explained data"""
      return 0
   
   def evaluate_anomalies(self):
      """Evaluates the amount of raised anomalies that are now explained
      
      Returns:
      - a float between 0 and 1, representing the ratio of explained anomalies"""
      return 0
   
   def __str__(self):
      """Return a report describing the interpretation process"""
      desc = ["Geological Interpretation Process:"]
      if self.dataset is None:
         desc+= ["|- Dataset: None"]
      else:
         desc+= ["|- Dataset: "+"\n| |".join(self.dataset.__str__().split("\n"))]
         
      if self.knowledge_framework is None:
         desc+= ["|- Geological Knowledge Framework: None"]
      else:
         desc+= ["|- "+"\n| |".join(self.knowledge_framework.__str__().split("\n"))]
         
      if self.representation_space is None:
         desc+= ["|- No representation space defined, this is an abstract interpretation only."]
      else:
         desc+= ["|- "+"\n| |".join(self.representation_space.__str__().split("\n"))]
         
      return "\n".join(desc)
   
   def run(self, max_iter= None):
      """Runs the iterative interpretation process
      
      Parameters:
      - max_iter (int): if set, it specifies the maximum number of iterations to run before terminating.
      Iteration (epoch) are numbered starting at 0, this way the counter also represents the number of passed iterations.
      """
      
      # run the iterative process as long as a termination criterion is not reached
      
      #first update the status to make sure it is up to date, and reset the termination criteria
      self.update_status()
      
      # new epoch increments the iteration count and check termination
      try:
         while self.new_epoch(max_iter = max_iter):
            
            # 1. Plan
            self.plan()
            print(self.situation.features)
            
            if len(self.situation.features) == 0:
               raise MalissiaBaseError("Empty feature selection")
            self.situation.features[0].is_Explained_by = [self.situation.features[0]]
            print(self.dataset.get_unexplained_observations())
            
            # 2. Do
            # -----------------------------
            # 2.1 execute the specified action
            
            # 3. Check
            # -----------------------------
            #  3.1 Evaluate internal consistency
            #  3.2 Evaluate relational likelihood
            #  3.3 Evaluate feature explanation
            
            # 4. Act
            # -----------------------------
            # 4.1 Generate anomalies and report
            # 4.2 update status
            
      except KeyboardInterrupt:
         self.status["termination_criterion"] = TerminationFlag.USER
         
      return self.status["termination_criterion"]
         
   def new_epoch(self, max_iter= None):
      """Starts a new epoch (iteration) unless termination criteria were reached
      
      Returns:
      - True if the new epoch should start, False if iterations should stop
      - max_iter (int): if set, it specifies the maximum number of iterations to run before terminating"""
      keep_going = True
      
      self.increment_epoch()
      if (max_iter is not None) and (self.status["epoch"] >= max_iter):
         self.status["termination_criterion"] = TerminationFlag.MAX_ITER
         return False
      
      return keep_going
      
   def increment_epoch(self):
      """Initialize or increment the epoch count"""
      if self.status["epoch"] is None:
         self.status["epoch"] = 0
      else:
         self.status["epoch"] += 1
      
   def select_strategy(self, key):
      """Select a the implementation to be used based on strategy"""
      if key not in self.strategies:
         raise MalissiaBaseError("The task has no defined strategy: " + key)
      strat = self.strategies[key]
      if isinstance(strat,list):
         strat = random.choice(strat)
      return strat
   
   def select_interpretation_move(self, key = None, random = False):
      """Select the type of interpretation action to be taken.
      
      Parameters:
      - key: the key corresponding to the chosen action, see `interpretation_moves`.
      This is to be used only for forcing the choice. Default, None.
      - random: if True, will be chosen among the available options (default: False).
      """
      
      # random of user choice
      if key is None and random:
         key = random.choice( self.interpretation_moves.keys() )
      if key is not None:
         return self.interpretation_moves[key]
      
      # default move is creating an new interpretation
      move = self.interpretation_moves["NewInterpretation"]
      # when applicable do others
      # if ...:
      #   move = self.interpretation_moves["UpdateInterpretation"]
      # elif ...:
      #   move = self.interpretation_moves["RemoveInterpretation"]
      return move
      
   def apply_new_interpretation(self, random_choice= False, debug= False):
      """Implements the creation of a new interpretation object"""
      if self.situation is None:
         raise MalissiaBaseError("Something needs to be selected for interpretation. Here the situtation is None.")
      
      # get a list of possibly explaining concepts for the features to be explained
      possible_explanations = [concept
                             for feature_i in self.situation.features
                             for concept in self.knowledge_framework.get_possible_interpretations_of(feature_i)]
      
      ## sort existing concepts by preference
      #possible_explanations = possible_explanations
      
      # pick an explaining concept
      candidate_explanation_class = random.choice(possible_explanations)
      
      # check for existing instances of this concept in the selected context
      existing_instance_mask = [self.knowledge_framework.isinstance(context_object, candidate_explanation_class)
                            for context_object in self.situation.context]
      # if applicable use it else create a new one
      if np.any(existing_instance_mask):
         existing_instances = self.situation.context[existing_instance_mask]
         self.situation.candidate_explaining_object = random.choice(existing_instances)
      else:
         constructor = self.knowledge_framework.generate_object_constructor(
                              candidate_explanation_class,
                              interpretation_situation = self.situation,
                              interpretation_status = self.status,
                              random_choice = random_choice,
                              debug= debug
                              )
         # Todo: catch the error when missing constructor and generate a report for next iteration
         self.situation.candidate_explaining_object = constructor(
                                                         interpretation_situation = self.situation,
                                                         interpretation_status = self.status
                                                         )
         self.register_interpreted_object(self.situation.candidate_explaining_object)
   
   def apply_update_interpretation(self):
      """Implements the update of an existing interpretation object"""
      pass
   
   def apply_remove_interpretation(self):
      """Implements the removal of an existing interpretation object"""
      self.knowledge_framework.remove_all_instances(self.situation.features)
   
      
   def plan(self):
      """Perform the Plan part of the algorithm
      
      1. Select a situation to be explained
      2. Select a Type of move to be performed in the interpretation process"""
      self.select_strategy("SituationSelection")(self)
      self.select_interpretation_move()

    

### Interpretation Process Tasks

In [ ]:


class RandomSituationStrategy(Strategy):
    """Strategy for randomly selecting a situation"""
    name = "RandomSituationStrategy"
    short_desc = "Random Situation Selection"
    full_desc = "This is implementing random selection of situation to be explained."
    strategy_type= StrategyType.RANDOM
    
    def __init__(self, **kargs):
        """Initialisation of radom situation selection"""
        super().__init__(**kargs)
        
    
    def check_applicability(task):
        """Checks if applicable
        
        i.e., for a SelectSituation
        """
        applicable = isinstance(task, SelectSituation)
        applicable &= task.context is not None
        return applicable
    
    def execute(self, task):
        """apply the strategy.
        
        Parameters:
        - task: the task for which this strategy must be applied"""
        
        feature_selection_task = task.context.strategies.generate_task(SelectObservation, strategy_type= StrategyType.RANDOM, context= task.context)
        selected_feature = feature_selection_task.execute(return_result= True)
        
        task.result = InterpretationSituation( selected_feature= selected_feature,  process= task.context)
  
class SelectSituation(ResultTask):
    """Task for selecting a situation to be explained (feature to be explained + interpretation context)"""
    
    name = "SelectSituation"
    short_desc = "selects a situation to ne explained"
    full_desc = "Selects features to be explained and an interpretation context."
    result_type = InterpretationSituation
    available_strategies = set((RandomSituationStrategy,))
    
    def __init__(self, context:GeologicalInterpretationProcess, feature_choices = None, n= None, **kargs):
        """Initialisation of a situation selection task
        
        Parameters:
        - feature_choices: a preselected list of features to be explained. If None (default), the dataset in the context will be used 
        - context: a `GeologicalInterpretationProcess`, as this is an action task, this context will be modified by the task.
        It will also be used to define the possible selection unless feature_choices is given.
        - n (int): number of features to be selected, if None, this is inferred from the strategies in the context."""
        self.feature_choices = feature_choices
        self.n = n
        super().__init__(context = context, result_type= SelectSituation.result_type, **kargs)
        
    

# Creation of a square surface using Imd method

In [ ]:
import numpy as np

def calculate_orientation(normal_vector):
    # Extract components of the normal vector and center point
    Nx, Ny, Nz = normal_vector

    # Calculate yaw angle (azimuth)
    yaw = math.atan2(Ny, Nx)

    # Calculate roll angle
    roll = math.atan2(math.sqrt(Nx**2 + Ny**2), Nz)

    # Calculate pitch angle
    pitch = math.atan2(-Nx, Ny)

    # Convert angles from radians to degrees if needed
    yaw_degrees = math.degrees(yaw)
    roll_degrees = math.degrees(roll)
    pitch_degrees = math.degrees(pitch)

    return yaw_degrees, roll_degrees, pitch_degrees

def construct_square_from_sad(normal_vector, center, 
                              side_length = None, area = None, diagonal=None):
    """function for constructing a square based on its normal vector, center point and 
    SAD (either his side length,diagonal or its area) """

    # Calculate orientation angles
    roll_degrees, pitch_degrees, yaw_degrees = calculate_orientation(normal_vector)

    # Calculate half-length of the square's side
    if side_length == None and area !=None:
        side_length = np.sqrt(area)
    elif side_length == None and diagonal !=None:
        side_length = diagonal/np.sqrt(2)
    elif side_length == None and area == None and diagonal == None :
        raise ValueError("please introduce value of a paramter to calculate geometery of the square")

    half_length = side_length / 2

    # Convert orientation angles from degrees to radians
    roll_rad = np.radians(roll_degrees)
    pitch_rad = np.radians(pitch_degrees)
    yaw_rad = np.radians(yaw_degrees)

    # Rotation matrix for the specified angles
    rotation_matrix = np.array([
        [np.cos(roll_rad) * np.cos(yaw_rad) - np.sin(roll_rad) * np.sin(pitch_rad) * np.sin(yaw_rad), -np.cos(roll_rad) * np.sin(yaw_rad) - np.sin(roll_rad) * np.sin(pitch_rad) * np.cos(yaw_rad), np.cos(pitch_rad) * np.sin(roll_rad)],
        [np.cos(pitch_rad) * np.sin(yaw_rad), np.cos(pitch_rad) * np.cos(yaw_rad), -np.sin(pitch_rad)],
        [np.sin(roll_rad) * np.cos(yaw_rad) + np.cos(roll_rad) * np.sin(pitch_rad) * np.sin(yaw_rad), np.cos(roll_rad) * np.sin(pitch_rad) * np.cos(yaw_rad) - np.sin(roll_rad) * np.sin(yaw_rad), np.cos(roll_rad) * np.cos(pitch_rad)]
    ])

    # Calculate the coordinates of the four vertices
    vertices = [
        center + np.dot(rotation_matrix, np.array([-half_length, -half_length, 0])),
        center + np.dot(rotation_matrix, np.array([half_length, -half_length, 0])),
        center + np.dot(rotation_matrix, np.array([half_length, half_length, 0])),
        center + np.dot(rotation_matrix, np.array([-half_length, half_length, 0]))
    ]

    # Define the edges (pairs of vertex indices)
    edges = [
        (0, 1),
        (1, 2),
        (2, 3),
        (3, 0)
    ]

    # Create a square as a collection of polygons
    square = [vertices[i] for i in [0, 1, 2, 3, 0]]

    return vertices, edges, square


# Tests

## Testing constructors

## Testing ontology manipulation

The knowledge manipulated in this package is formalised in an ontology,<br>
which is store in a *.owl* file.

It is named **MOGI** for **M**inimal **O**ntology for **G**eological **I**nterpretation

To manipulated this ontology, we use the package **owlready2** available from here: https://owlready2.readthedocs.io

Ontology provides access to its components, e.g.:
* classes
* properties
* individuals
* rules

In [ ]:
print(list(mogi().classes()))
print(list(mogi().properties()))
print(list(mogi().individuals()))
print(list(mogi().rules()))

More specific elements can be searched through simple queries:

In [ ]:
mogi().search(iri = "*Surface*")

In [ ]:
mogi.search(type= mogi().PointBased_Observation, qualities= ["dip","occurrence"])

In [ ]:
mogi.search(type= mogi().PointBased_Observation, qualities= {"dip":45})

In [ ]:
mogi.search(qualities= {"dip":45})

In [ ]:
mogi().search(name="o?")

In [ ]:
mogi().search(name="o1")

In [ ]:
mogi._ontology_backend.Thing

### Reasoner

Ontologies are even more powerful thansk to their capabilities to use reasoning for infering types, properties, and relationships that were not explicitly stated.
This is usefull for obtaining results implied by the already stated information.

This is achieved by running a *reasoner* on the ontology as follows.

## Testing GeologicalKnowledgeManager and ontology manipulation 

##### when the ontoology is attached to the knowledge manager, it is now referred to as mogi() and the roginal backend (owl or others) is now referred to as _ontology_backend


In [ ]:
GeologicalKnowledgeManager().load_knowledge_framework()
print(GeologicalKnowledgeManager().get_knowledge_framework())

In our approach, geological datasets will be progressively interpreted in terms of structural objects,<br>
based on a formal definition of concepts own by a **GeologicalKnowledgeManager**.<br>


In [ ]:
mogi = GeologicalKnowledgeManager().get_knowledge_framework()
mogi.sync_reasoner()
mogi.name

In [ ]:
mogi().classes()

In [ ]:
mogi().Stratigraphic_Part.is_Possible_Explanation_Of

In [ ]:
mogi().Stratigraphic_Part.has_Representation

In [ ]:
mogi.get_objects_potentially_explained_by(mogi().Stratigraphic_Surface)

In [ ]:
mogi().search(type= mogi().PointBased_Observation)

## Testing Representation space

In [ ]:
space= PhysicalRepresentationSpace(2)

space.set_extension_from_data(padding= None)

print(space)


In [ ]:
space= PhysicalRepresentationSpace(3)
space.compute_line_attitude_from_two_points([0,0,0],[1,-1,-1])

In [ ]:
p = [[0.,0,0],[-1,0,-1],[0,-1,0],[-1,-1,-1.3]]
space= PhysicalRepresentationSpace(3)
plane = space.compute_attitude_from_points(p)
plane

In [ ]:
dip = 90
dip_dir = 270
space= PhysicalRepresentationSpace(3)
space.compute_normal_from_dip_dir(dip, dip_dir)


In [ ]:
space= PhysicalRepresentationSpace(3)
space.compute_dip_dir_from_normal([-1,1,-1])


In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(pv.PolyData(p))
plotter.add_mesh(pv.PolyData(plane["center"]), color="red")

plotter.add_arrows(plane["center"],plane["major_axis"], color= "red")
plotter.add_arrows(plane["center"],plane["minor_axis"], color= "green")
plotter.add_arrows(plane["center"],plane["normal"], color= "blue")
plotter.show_axes()
plotter.show(cpos='xz', window_size = (600,400))

In [ ]:
plotter = pv.Plotter()
center = np.array([0,0,0])
plotter.add_arrows(np.repeat([center],4,axis=0), np.array(p))
plane = space.compute_principal_directions(p)
average = space.compute_average_vector(p)
plotter.add_arrows(center, average, color="Black")

#plotter.add_arrows(plane["center"],plane["vectors"][0], color= "red")
#plotter.add_arrows(plane["center"],plane["vectors"][1], color= "green")
#plotter.add_arrows(plane["center"],plane["vectors"][2], color= "blue")
plotter.show_axes()
plotter.show(cpos='xz', window_size = (600,400))

In [ ]:
plane

In [ ]:
space = PhysicalRepresentationSpace(coordinate_labels= "depth")

In [ ]:
space.compute_line_attitude_from_two_points([0,0,0],[-10,0,-2])

In [ ]:
try: PhysicalRepresentationSpace()
except Exception as e:
    print(e)

In [ ]:
space= PhysicalRepresentationSpace(coordinate_labels=["X","Y"])
print(space)

In [ ]:
space= PhysicalRepresentationSpace(coordinate_labels=["X","Z"])
print(space)

## Testing DataSet & Manual entries

In [ ]:
obs = mogi().search(type=mogi().PointBased_Observation)

In [ ]:
if len(obs) > 0 : 
    mogi.show_instance_qualities(obs[0])

In [ ]:
dataset= GeologicalDataset()
dataset.remove_all_observations()
print(dataset)

In [ ]:
dataset.head()

In [ ]:
dataset.get_observations()

In [ ]:
mogi.show_all_instance_qualities(dataset.get_observations())

In [ ]:
dataset.remove_observation_by_name("D8")
dataset.get_observations()

In [ ]:
dataset.remove_all_observations()
print(dataset)

In [ ]:

physical_space = PhysicalRepresentationSpace(coordinate_labels= ["x","y","z"])
dataset.setup_representation_space(physical_space= physical_space)
dataset.add_occurrence_observation(name="DD", observed_object= "Keuper", x= 1, z= 2, y= 0, occurrence= True)
dataset.add_occurrence_observation(name="DN", observed_object= "Keuper",  x= 3, z= 1, y= 0)
dataset.update_extension()
print(dataset)

dataset.get_observations()

In [ ]:
dataset.head()

In [ ]:
dataset.add_orientation_observation(name="DO", observed_object= "Trias", dip= 30, dip_dir= 270, x= 2, z= 2, y= 0)
print(dataset)

dataset.get_observations()

In [ ]:
dataset.head()

### From file

In [ ]:
dataset = load_dataset_from_csv("../inputs/data_for_paper.csv", sep=";", index= "Id", coordinate_labels=["x","y","z"], labels={"ID":"Id", "strike":"dip_dir","name":"observed_object"})
print(dataset)

In [ ]:
dataset.head()

In [ ]:
# illustrates errors because of wrong separator
try:
    dataset2= load_dataset_from_csv("../inputs/data_for_paper.csv", labels={"ID":"Id", "strike":"dip_dir","name":"observed_object"})
except Exception as e:
    print(e)
    gkf = GeologicalKnowledgeManager().get_knowledge_framework()
    observations = gkf.search(type=gkf().PointBased_Observation)
    gkf.remove_all_instances(observations)

In [ ]:
# illustrates errors when leaving wrong labels
try:
    dataset3 = load_dataset_from_csv("../inputs/data_for_paper.csv", sep=";", index= "ID")
except Exception as e:
    print(e)
    gkf = GeologicalKnowledgeManager().get_knowledge_framework()
    observations = gkf.search(type=gkf().PointBased_Observation)
    gkf.remove_all_instances(observations)

### Manual creation

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_head = np.array(['name', 'x', 'y', 'z', 'dip_dir', 'dip', 'observed_object','occurrence'])
data_array = np.array([['D1', 15, 20, 35, 270, 45, 'Trias_Base',True],
                       ['D2', 30, 25, 50, 270, 45, 'Trias_Base',True],
                       ['D3', 60, 30, 40, 90, 45, 'Trias_Base',True],
                       ['D4', 75, 15, 25, 90, 45, 'Trias_Base',True],
                       ['D5', 110, 20, 40, 270, 63, 'Trias_Base',True],
                       ['D6', 120, 20, 60, 270, 64, 'Trias_Base',True],
                       ['D7', 155, 20, 60, 89, 39, 'Trias_Base',True],
                       ['D8', 190, 20, 30, 91, 40, 'Trias_Base',True],
                       ['D11', 25, 22, 45, np.nan, np.nan, None ,True],
                       ['D22', 50, 22, 50, np.nan, np.nan, None,True],
                       ['D44', 100, 30, 20, np.nan, np.nan, None,True],
                       ['D77', 168, 30, 47, np.nan, np.nan, None,True]]
)
data_test = pd.DataFrame(data = data_array, columns = data_head)
data_test = data_test.astype({'name':str, 'x':float, 'y':float, 'z':float, 'dip_dir':float, 'dip':float, 'observed_object':str, 'occurrence':bool})
data_test.set_index("name", inplace = True)
data_test

In [ ]:
dataset = load_dataset_from_dataframe(data_test, coordinate_labels= ["x","y","z"])
print(dataset)

In [ ]:
dataset.head()

In [ ]:
from IPython.display import display, HTML

chart = HTML(dataset.info().replace("\n","<br>"))
display(chart)

chart = HTML(dataset.to_dataframe().to_html())
display(chart)


## Testing visualizations & drawings 

In [ ]:
draw_line([0,0],30, "left")
draw_dip_symbol([0,1],60, "right", polarity= "up", color= "red" )
plt.gca().set_aspect("equal")

#### Demo

In [ ]:
physical_space = PhysicalRepresentationSpace(coordinate_labels=["x","y","z"])
dataset= GeologicalDataset(physical_space=physical_space)
print(dataset) 

In [ ]:
section = AxisAlignedCrossSection(physical_space)
section.draw_dip_symbol([1,30],30, "right", polarity= "up", length=10 )
section.draw_occurrence_symbol([100,40])

In [ ]:
section = AxisAlignedCrossSection(physical_space)
section.show()

## Testing objets constructors

In [ ]:
from scipy.spatial.transform import Rotation
dip = 60
dip_dir = 45
Rotation.from_euler("XZY",[dip,dip_dir,0], degrees= True).as_matrix()[-1]

In [ ]:
Rotation.from_euler("XZY",[[60,0,0],[60,180,0]], degrees= True).mean().as_euler("XZY",degrees= True)

In [ ]:
Rotation.from_euler("XZY",[[60,0,0],[60,180,0]], degrees= True).mean().as_matrix()[-1]

## Testing Tasks

In [ ]:
task = __default_strategy_factory__.generate_task(ExampleTask, strategy_type= StrategyType.RANDOM)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)

In [ ]:
task = __default_strategy_factory__.generate_task(ExampleTask)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)

In [ ]:
try:
    strat = Strategy()
    strat.check_applicability()
except MalissiaBaseError as e:
    print(e)

In [ ]:
task = RandomExampleTask()
if task.check_applicability():
    task.execute()
task.result

In [ ]:
task = ActionTask(None)
task.name

In [ ]:
try:
    task.execute()
except MalissiaBaseError as e:
    print(e)

In [ ]:
ExampleTask.available_strategies

In [ ]:
UserExampleTask.available_strategies

In [ ]:
ActionTask.available_strategies

In [ ]:
task = UserExampleTask()
if task.check_applicability():
    task.execute()
task.result

In [ ]:
task = __default_strategy_factory__.generate_task(SelectionTask, choices = [1,2,27,42])
task.execute()
if isinstance(task,ResultTask):
    print(task.result)

In [ ]:
choices = dataset.get_observations()
task = __default_strategy_factory__.generate_task(SelectionTask, choices = choices, strategy_type= StrategyType.RANDOM)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)
print("Result type:",type(task.result))

In [ ]:
task = __default_strategy_factory__.generate_task(SelectObservation, dataset = dataset)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)
print("Result type:",type(task.result))

In [ ]:
task = __default_strategy_factory__.generate_task(SelectSituation, context = gip)
task.execute()
if isinstance(task,ResultTask):
    print(task.result)
print("Result type:",type(task.result))

## Testing the creation of the different possible instances

In [ ]:
GeologicalKnowledgeManager().load_knowledge_framework()
mogi = GeologicalKnowledgeManager().get_knowledge_framework()
# clean all individuals
mogi.remove_all_instances()
mogi.sync_reasoner()
mogi.name

In [ ]:
mogi.show_all_instance_qualities()

### Points

In [ ]:
point = constructor_point(mogi, ["X","Y","Z"], [1,2,3], name= "point_test")
mogi.show_instance_qualities(point)

In [ ]:
mogi.sync_reasoner()

### Vectors

In [ ]:
vect = constructor_vector(mogi, ["X","Y","Z"], [1,2,3], name= "vector_test")
mogi.show_instance_qualities(vect)

In [ ]:
mogi.sync_reasoner()

### Observations

In [ ]:
test_obs_occur = constructor_observation(mogi, name= "obs_occurrence", coord_labels= ["X","Y","Z"], X= 1, Y= 2, Z= 3, occurrence= True)
mogi.show_instance_qualities(test_obs_occur)
mogi.show_instance_qualities(test_obs_occur.has_Center[0])

In [ ]:
test_obs_dip = constructor_observation(mogi, name= "obs_dip", coord_labels= ["X","Y","Z"], X= 1, Y= 2, Z= 3,
                                       dip= 30, dip_dir = 270, polarity= True, size= 0.2)
mogi.show_instance_qualities(test_obs_dip)
mogi.show_instance_qualities(test_obs_dip.has_Center[0])

In [ ]:
mogi.sync_reasoner()

### Planar_Surface

In [ ]:
nodes = [
    constructor_point(mogi, ["X","Y","Z"],[0,0,0], name="N0"),
    constructor_point(mogi, ["X","Y","Z"],[1,0,1], name="N1"),
    constructor_point(mogi, ["X","Y","Z"],[1,1,1], name="N2"),
    constructor_point(mogi, ["X","Y","Z"],[0,1,0], name="N3")
    ]
plan = constructor_planar_surface_from_nodes(mogi,nodes, name="plan_from_nodes")
mogi.show_instance_qualities(plan)
mogi.show_instance_qualities(plan.has_Center[0])
mogi.show_instance_qualities(plan.has_Normal[0])


In [ ]:
coord_labels = ["X","Y","Z"]
coords = [
    [0,0,0],
    [1,0,1],
    [1,1,1],
    [0,1,0]
    ]
plan = constructor_planar_surface_from_coords(mogi, coord_labels= coord_labels, coords= coords, name= "plan_from_coord")
mogi.show_instance_qualities(plan)
mogi.show_instance_qualities(plan.has_Center[0])
mogi.show_instance_qualities(plan.has_Normal[0])


In [ ]:
coord_labels = ["X","Y","Z"]
center = [0.,0.,0.]
plan = constructor_planar_surface_from_center_attitude(mogi, coord_labels= coord_labels, center= center,
                                                       size= 2, dip= 30, dip_dir= 90, polarity= 1,
                                                       name= "plan_from_attitude"
                                                       )
mogi.show_instance_qualities(plan)
mogi.show_instance_qualities(plan.has_Center[0])
mogi.show_instance_qualities(plan.has_Normal[0])


In [ ]:
mogi.sync_reasoner()

### Stratigraphy

In [ ]:
physical_space = PhysicalRepresentationSpace(3)
points = [mogi().N0, mogi().N1, mogi().N2, mogi().N3]
#mogi.show_all_instance_qualities(points)
#print(np.array([physical_space.get_object_coordinates(feature_i) for feature_i in points]))
location_constructor([mogi().N0, mogi().N1, mogi().N2, mogi().N3], physical_space= physical_space, method= "random", return_as_dict= False)

In [ ]:
physical_space = PhysicalRepresentationSpace(3)
points = [mogi().N0, mogi().N1, mogi().N2, mogi().N3]
#mogi.show_all_instance_qualities(points)
#print(np.array([physical_space.get_object_coordinates(feature_i) for feature_i in points]))
attitude_constructor(points, physical_space= physical_space, knowledge_framework= mogi, method= "average")

In [ ]:
physical_space = PhysicalRepresentationSpace(3)
points = [mogi().N0, mogi().N2]
#mogi.show_all_instance_qualities(points)
#print(np.array([physical_space.get_object_coordinates(feature_i) for feature_i in points]))
attitude_constructor(points, physical_space= physical_space, knowledge_framework= mogi, method= "average")

In [ ]:
physical_space = PhysicalRepresentationSpace(coordinate_labels= ["X","Y","Z"])
obs = []
obs += [constructor_observation(mogi, name= "O1", coord_labels= ["X","Y","Z"], X= 1, Y= 2, Z= 3,
                                       dip= 30, dip_dir = 270, polarity= True, size= 0.2)]
#mogi.show_all_instance_qualities(points)
#print(np.array([physical_space.get_object_coordinates(feature_i) for feature_i in points]))
attitude_constructor(obs, physical_space= physical_space, knowledge_framework= mogi, method= "average")

In [ ]:
physical_space = PhysicalRepresentationSpace(coordinate_labels= ["X","Y","Z"])
obs = []
obs += [constructor_observation(mogi, name= "O1", coord_labels= ["X","Y","Z"], X= 1, Y= 2, Z= 3,
                                       dip= 30, dip_dir = 270, polarity= True, size= 0.2)]
obs += [constructor_observation(mogi, name= "O2", coord_labels= ["X","Y","Z"], X= 2, Y= 2, Z= 3,
                                       dip= 31, dip_dir = 90, polarity= True, size= 0.2)]
#mogi.show_all_instance_qualities(points)
#print(np.array([physical_space.get_object_coordinates(feature_i) for feature_i in points]))
attitude_constructor(obs, physical_space= physical_space, knowledge_framework= mogi, method= "average")

In [ ]:
mogi.show_all_instance_qualities()

In [ ]:
mogi.sync_reasoner()

### testing anomalies

In [ ]:
stratigraphicpart1 = mogi.Stratigraphic_Part()
stratigraphicpart2 = mogi.Stratigraphic_Part()
anom3 = discontinuousStratigaphyAnomaly_constructor(knowledge_framework= mogi, stratigraphicpart2 = stratigraphicpart2, stratigraphicpart1 = stratigraphicpart1)

In [ ]:
anom3.is_Related_To

## Testing interpretations

In [ ]:
obs = mogi().search(type= mogi().PointBased_Observation)

In [ ]:
if len(obs) > 0:
    interpretations = mogi.get_possible_interpretations_of(obs[0])
interpretations

In [ ]:
interpretations[0].has_Representation

In [ ]:
mogi.get_objects_potentially_explained_by(mogi().Stratigraphic_Part)

In [ ]:
if len(obs) > 1:
    interpretations = mogi.get_possible_interpretations_of(obs[1])
interpretations

In [ ]:
mogi().Stratigraphic_Surface.is_Possible_Explanation_Of

In [ ]:
gip = GeologicalInterpretationProcess(dataset)

In [ ]:
gip.run(max_iter=7)

In [ ]:
print(gip)

In [ ]:
gip = GeologicalInterpretationProcess(dataset, strategies= {"SituationSelection":select_user_unexplained_feature} )
gip.run(max_iter=2)

In [ ]:
gip.strategies

#### Testing interpretation from features

In [ ]:
GeologicalKnowledgeManager().load_knowledge_framework()
mogi = GeologicalKnowledgeManager().get_knowledge_framework()
mogi.remove_all_instances()
mogi.sync_reasoner()
mogi.name

In [ ]:
data_head = np.array(['name', 'x', 'y', 'z', 'dip_dir', 'dip', 'observed_object','occurrence'])
data_array = np.array([['D1', 15, 20, 35, 270, 45, 'Trias_Base',True],
                       ['D2', 30, 25, 50, 270, 45, 'Trias_Base',True],
                       ['D3', 60, 30, 40, 90, 45, 'Trias_Base',True],
                       ['D4', 75, 15, 25, 90, 45, 'Trias_Base',True],
                       ['D5', 110, 20, 40, 270, 63, 'Trias_Base',True],
                       ['D6', 120, 20, 60, 270, 64, 'Trias_Base',True],
                       ['D7', 155, 20, 60, 89, 39, 'Trias_Base',True],
                       ['D8', 190, 20, 30, 91, 40, 'Trias_Base',True],
                       ['D11', 25, 22, 45, np.nan, np.nan, None ,True],
                       ['D22', 50, 22, 50, np.nan, np.nan, None,True],
                       ['D44', 100, 30, 20, np.nan, np.nan, None,True],
                       ['D77', 168, 30, 47, np.nan, np.nan, None,True]]
)
data_test = pd.DataFrame(data = data_array, columns = data_head)
data_test = data_test.astype({'name':str, 'x':float, 'y':float, 'z':float, 'dip_dir':float, 'dip':float, 'observed_object':str, 'occurrence':bool})
data_test.set_index("name", inplace = True)
dataset = load_dataset_from_dataframe(data_test, coordinate_labels= ["x","y","z"])
print(dataset)

In [ ]:
dataset.head()

In [ ]:
section = AxisAlignedCrossSection(dataset.physical_space)
section.show()

In [ ]:
situation = InterpretationSituation(
    [mogi().D1, mogi().D2, mogi().D4,mogi().D5, mogi().D3]
)
print(situation.features)
mogi.show_all_instance_qualities(situation.features)

In [ ]:
interp = constructor_surface_part_from_interpretation(knowledge_framework= mogi, interpretation_situation= situation,
                                             physical_space= dataset.physical_space, name= "interp_surf")
mogi.show_instance_qualities(interp)
mogi.show_instance_qualities(interp.has_Representation[0])
mogi.show_all_instance_qualities(interp.explain)

In [ ]:
section = AxisAlignedCrossSection(dataset.physical_space)
section.show()

drawing_function = section.get_drawing_method(interp.is_instance_of[0])
drawing_function(interp, section, setup_drawing= False)
section.get_drawing_method(InterpretationSituation)(situation, section)

In [ ]:
section.get_drawing_method(InterpretationSituation)(situation, section)